<a href="https://colab.research.google.com/github/nihilistau/ComfyUI-Basic/blob/main/comfyui_persistent_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab-Coder-Expert Persona

You are "Colab-Coder-Expert", an AI that generates production-ready, idempotent, and modular Colab notebook cells for running ComfyUI with a tunnel. Run cells sequentially. Follow the constraints: mount Drive to /content/drive/MyDrive, use Drive for persistent installs, add checks to avoid re-running installations, use Colab forms for user inputs, and run background processes with nohup and PID files.

In [2]:
#@title Mount Google Drive (idempotent)
from google.colab import drive
import os, sys, time

#@markdown Select a Drive base path under your MyDrive to install ComfyUI and related files:
drive_subpath = '/comfyui_env'  #@param {type:'string'}
base_drive_path = os.path.join('/content/drive/MyDrive', drive_subpath.strip('/'))

def mount_drive(path='/content/drive'):
    if os.path.exists('/content/drive') and os.path.ismount('/content/drive'):
        print('Google Drive already mounted.')
        return True
    print('Mounting Google Drive...')
    drive.mount('/content/drive')
    # slight wait for mount to settle
    for _ in range(5):
        if os.path.exists('/content/drive/MyDrive'):
            break
        time.sleep(0.5)
    return os.path.exists('/content/drive/MyDrive')

ok = mount_drive()
if not ok:
    raise SystemExit('Failed to mount Google Drive. Make sure you complete the auth popup.')

# Ensure base path exists
os.makedirs(base_drive_path, exist_ok=True)
print(f'Using drive path: {base_drive_path}')

# Create subfolders for repo, logs, pids
for d in ['repo', 'logs', 'pids']:
    p = os.path.join(base_drive_path, d)
    os.makedirs(p, exist_ok=True)
    print('Ensured', p)

# Expose variables for later cells
DRIVE_ROOT = /content/drive/MyDrive/comfy_env/ComfyUI
DRIVE_BASE = base_drive_path
REPO_PATH = os.path.join(DRIVE_BASE, 'repo')
LOGS_PATH = os.path.join(DRIVE_BASE, 'logs')
PIDS_PATH = os.path.join(DRIVE_BASE, 'pids')
print('Environment variables set: DRIVE_BASE, REPO_PATH, LOGS_PATH, PIDS_PATH')

SyntaxError: invalid syntax (ipython-input-2881638174.py, line 37)

# ComfyUI Persistent Workspace
<a href="https://colab.research.google.com/github/your-org/Collab-Manager-Remastered/blob/main/notebooks/comfyui_persistent_setup.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
This notebook provisions and operates a long-lived ComfyUI environment backed by Google Drive. Each section below focuses on a different part of the workflow: setup actions, service launch, and runtime control. Run the cells in order and revisit any section as needed.
*Tip:* Update the badge link above to point at your repository before sharing this notebook publicly.

## 1. Setup & Dependency Management
Use the interactive checklist to create or repair the persistent virtual environment, pull ComfyUI updates, and refresh custom node dependencies. This cell can be rerun safely whenever you modify the workspace contents.

In [2]:
import hashlib
import json
import os
import shutil
import subprocess
import sys
import time
import zipfile
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display

STEP_ESTIMATES = {
    'initial_install': '~3-6 min (clone and base dependencies)',
    'install_dependencies': '~1-3 min (reinstall requirements)',
    'check_dependencies': '~10-25 s (pip health scan)',
    'update_comfyui': '~20-60 s (git pull)',
    'update_manager': '~1-3 min (manager dependencies)',
    'update_custom_nodes': '~1-6 min (per node requirements)',
    'check_pip_tools': '~8-20 s (pip status)',
    'update_pip_tools': '~15-45 s (pip/setuptools/wheel upgrade)',
    'check_python_runtime': '~6-15 s (runtime probe)',
    'repair_python_runtime': '~20-50 s (runtime fixes)',
    'check_configs': '~5-15 s (config audit)',
    'repair_configs': '~5-20 s (config placeholders)',
    'check_cloudflared_dependencies': '~5-15 s (environment probe)',
    'install_cloudflared': '~20-40 s (download binary)',
    'update_cloudflared': '~20-40 s (refresh binary)',
    'scan_installation': '~10-40 s (validate file map)',
    'check_integrity': '~10-30 s (read-only scan)',
    'repair_missing': '~1-4 min (targeted repairs)',
    'augment_baseline': '~5-15 s (capture new baseline)',
    'rehydrate_environment': '~2-5 min (restore environment)',
}

BASE_DIR = Path('/content/drive/MyDrive/comfyui_env')
COMFY_DIR = BASE_DIR / 'ComfyUI'
CUSTOM_NODES_DIR = COMFY_DIR / 'custom_nodes'
VENV_DIR = BASE_DIR / 'venv'
BIN_DIR = BASE_DIR / 'bin'
LOG_DIR = BASE_DIR / 'logs'
RUNTIME_DIR = BASE_DIR / 'runtime'
INTEGRITY_STATE_FILE = RUNTIME_DIR / 'integrity_state.json'
CUSTOM_PATHS_FILE = RUNTIME_DIR / 'integrity_custom_paths.json'
PYTHON_BIN = VENV_DIR / 'bin' / 'python'
PIP_BIN = VENV_DIR / 'bin' / 'pip'
CLOUDFLARED_URL = 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64'
CLOUDFLARED_BIN = BIN_DIR / 'cloudflared'
MANAGER_REPO_URL = 'https://github.com/ltdrdata/ComfyUI-Manager.git'
MANAGER_DIR = CUSTOM_NODES_DIR / 'ComfyUI-Manager'
FILES_SNAPSHOT_LIMIT = 400

INTEGRITY_TARGETS = {
    'comfy_repo': {
        'path': COMFY_DIR,
        'type': 'dir',
        'description': 'ComfyUI repository checkout',
        'sentinel': COMFY_DIR / 'main.py',
        'expected_files': ['main.py', 'requirements.txt', 'web'],
    },
    'python_binary': {
        'path': PYTHON_BIN,
        'type': 'file',
        'description': 'Virtual environment python',
    },
    'pip_binary': {
        'path': PIP_BIN,
        'type': 'file',
        'description': 'Virtual environment pip entry point',
    },
    'cloudflared': {
        'path': CLOUDFLARED_BIN,
        'type': 'file',
        'description': 'Cloudflared tunnel binary',
    },
    'manager_repo': {
        'path': MANAGER_DIR,
        'type': 'dir',
        'description': 'ComfyUI Manager custom node',
        'sentinel': MANAGER_DIR / 'requirements.txt',
        'expected_files': ['requirements.txt'],
    },
}

PACKAGE_EXPECTATIONS = {
    'workspace_core': {
        'label': 'Workspace core',
        'items': [
            {'path': COMFY_DIR / 'main.py', 'type': 'file'},
            {'path': COMFY_DIR / 'requirements.txt', 'type': 'file'},
            {'path': VENV_DIR / 'bin' / 'python', 'type': 'file'},
            {'path': VENV_DIR / 'bin' / 'pip', 'type': 'file'},
            {'path': BIN_DIR / 'cloudflared', 'type': 'file'},
        ],
    },
    'manager_package': {
        'label': 'ComfyUI Manager',
        'items': [
            {'path': MANAGER_DIR, 'type': 'dir'},
            {'path': MANAGER_DIR / 'requirements.txt', 'type': 'file'},
        ],
    },
    'runtime_assets': {
        'label': 'Runtime assets',
        'items': [
            {'path': LOG_DIR, 'type': 'dir'},
            {'path': RUNTIME_DIR, 'type': 'dir'},
        ],
    },
}

CONFIG_EXPECTATIONS = [
    {'path': COMFY_DIR / 'config.json', 'type': 'file', 'description': 'ComfyUI global config', 'optional': True},
    {'path': COMFY_DIR / 'user', 'type': 'dir', 'description': 'User overrides directory', 'optional': True},
    {'path': COMFY_DIR / 'models', 'type': 'dir', 'description': 'Models directory', 'optional': True},
    {'path': CUSTOM_NODES_DIR, 'type': 'dir', 'description': 'Custom nodes directory', 'optional': False},
]

CONFIG_PLACEHOLDERS = {
    COMFY_DIR / 'config.json': '{\n  "__comment": "Generated placeholder config. Customize as needed."\n}\n',
    COMFY_DIR / 'user' / 'default.json': '{\n  "__comment": "User overrides go here."\n}\n',
}

ACTION_LOG_WIDGET = None


def log(message):
    target = globals().get('ACTION_LOG_WIDGET')
    if target is None:
        print(message)
    else:
        with target:
            print(message)


def ensure_directories():
    for path in (BASE_DIR, LOG_DIR, RUNTIME_DIR, BIN_DIR):
        path.mkdir(parents=True, exist_ok=True)


def run_command(command, cwd=None, env=None):
    joined = ' '.join(str(part) for part in command)
    log(f'$ {joined}')
    subprocess.run(command, cwd=str(cwd) if cwd else None, env=env, check=True)


def ensure_comfy_repo():
    ensure_directories()
    if COMFY_DIR.exists():
        log('ComfyUI repository already present.')
        return
    run_command(['git', 'clone', '--depth', '1', 'https://github.com/comfyanonymous/ComfyUI.git', str(COMFY_DIR)])


def create_virtualenv():
    ensure_directories()
    if not VENV_DIR.exists():
        run_command([sys.executable, '-m', 'venv', str(VENV_DIR)])
    else:
        log('Virtual environment already exists.')
    if not PYTHON_BIN.exists():
        raise RuntimeError('Virtual environment python missing after creation.')
    run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '--upgrade', 'pip'])
    requirements = COMFY_DIR / 'requirements.txt'
    if requirements.exists():
        run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '-r', str(requirements)])
    else:
        log('requirements.txt not found; skipping base dependency install.')


def download_cloudflared(force=False):
    ensure_directories()
    if CLOUDFLARED_BIN.exists() and not force:
        log('cloudflared binary already present; verifying permissions.')
        os.chmod(CLOUDFLARED_BIN, 0o755)
        return
    log('Downloading cloudflared binary...')
    run_command(['wget', '-q', '-O', str(CLOUDFLARED_BIN), CLOUDFLARED_URL])
    os.chmod(CLOUDFLARED_BIN, 0o755)
    log('cloudflared ready in persistent storage.')


def ensure_manager_repo(update=True):
    ensure_directories()
    CUSTOM_NODES_DIR.mkdir(parents=True, exist_ok=True)
    if MANAGER_DIR.exists():
        if update:
            run_command(['git', '-C', str(MANAGER_DIR), 'pull', '--ff-only'])
        else:
            log('ComfyUI Manager already present.')
    else:
        run_command(['git', 'clone', '--depth', '1', MANAGER_REPO_URL, str(MANAGER_DIR)])
    req_file = MANAGER_DIR / 'requirements.txt'
    if PYTHON_BIN.exists() and req_file.exists():
        run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '-r', str(req_file)])
    elif not PYTHON_BIN.exists():
        log('Virtual environment missing; manager dependencies deferred until initial install completes.')
    else:
        log('Manager requirements.txt not found; nothing to install.')


def refresh_custom_nodes():
    if not PYTHON_BIN.exists():
        raise RuntimeError('Virtual environment python missing; run initial install first.')
    if not CUSTOM_NODES_DIR.exists():
        log('No custom_nodes directory present; nothing to refresh.')
        return
    processed = 0
    for entry in sorted(CUSTOM_NODES_DIR.iterdir()):
        if not entry.is_dir():
            continue
        req_file = entry / 'requirements.txt'
        if req_file.exists():
            processed += 1
            log(f'Installing custom node dependencies: {entry.name}')
            run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '-r', str(req_file)])
    if not processed:
        log('No custom node requirements found; skipping dependency refresh.')


def initial_install():
    ensure_comfy_repo()
    create_virtualenv()
    download_cloudflared(force=True)
    ensure_manager_repo(update=False)
    refresh_custom_nodes()
    log('Initial install complete.')


def install_dependencies():
    ensure_directories()
    if not COMFY_DIR.exists():
        raise RuntimeError('ComfyUI directory missing; run initial install first.')
    if not PYTHON_BIN.exists():
        log('Virtual environment not found; creating now.')
        create_virtualenv()
    else:
        requirements = COMFY_DIR / 'requirements.txt'
        if requirements.exists():
            run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '-r', str(requirements)])
        else:
            log('requirements.txt not found; skipping base dependency install.')
    ensure_manager_repo(update=True)
    refresh_custom_nodes()
    log('Dependency installation refresh complete.')


def check_dependencies():
    if not PYTHON_BIN.exists():
        raise RuntimeError('Virtual environment python missing; run initial install first.')
    log('Running pip check to validate installed packages...')
    result = subprocess.run([str(PYTHON_BIN), '-m', 'pip', 'check'], capture_output=True, text=True)
    if result.stdout.strip():
        log(result.stdout.strip())
    if result.stderr.strip():
        log(result.stderr.strip())
    if result.returncode == 0:
        log('No dependency conflicts detected.')
    else:
        log(f'pip check exited with code {result.returncode}; review the messages above.')


def update_comfyui():
    if not COMFY_DIR.exists():
        raise RuntimeError('ComfyUI directory missing; run initial install first.')
    run_command(['git', '-C', str(COMFY_DIR), 'pull', '--ff-only'])
    log('ComfyUI repository updated.')


def update_manager():
    ensure_manager_repo(update=True)
    log('ComfyUI Manager refreshed.')


def update_custom_nodes():
    refresh_custom_nodes()
    log('Custom node requirements refreshed.')


def check_pip_tools():
    if not PYTHON_BIN.exists():
        raise RuntimeError('Virtual environment python missing; run initial install first.')
    version = subprocess.run([str(PYTHON_BIN), '-m', 'pip', '--version'], capture_output=True, text=True)
    log(version.stdout.strip() or 'pip --version produced no output.')
    outdated = subprocess.run([str(PYTHON_BIN), '-m', 'pip', 'list', '--outdated', '--format=freeze'], capture_output=True, text=True)
    lines = [line for line in outdated.stdout.strip().splitlines() if line]
    if lines:
        log('Outdated packages (showing up to 12 entries):')
        for entry in lines[:12]:
            log(f'  - {entry}')
        if len(lines) > 12:
            log(f'  ... {len(lines) - 12} additional packages reported by pip.')
    else:
        log('No outdated packages reported by pip.')
    return lines


def update_pip_tools():
    if not PYTHON_BIN.exists():
        create_virtualenv()
    log('Upgrading pip, setuptools, wheel, and packaging...')
    run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '--upgrade', 'pip', 'setuptools', 'wheel', 'packaging'])
    log('Core packaging tools updated.')


def _python_runtime_probe():
    script = (
        'import importlib, json, sys\n'
        'info = {"python_version": sys.version.split(" ")[0]}\n'
        'for name in ("wheel", "setuptools", "torch"):\n'
        '    data = {"available": False}\n'
        '    try:\n'
        '        module = importlib.import_module(name)\n'
        '        data["available"] = True\n'
        '        data["version"] = getattr(module, "__version__", None)\n'
        '        if name == "torch":\n'
        '            cuda = getattr(module, "cuda", None)\n'
        '            if cuda and hasattr(cuda, "is_available"):\n'
        '                data["cuda_available"] = bool(cuda.is_available())\n'
        '    except Exception as exc:\n'
        '        data["error"] = str(exc)\n'
        '    info[name] = data\n'
        'print(json.dumps(info))\n'
    )
    result = subprocess.run([str(PYTHON_BIN), '-c', script], capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(result.stderr.strip() or 'Failed to probe python runtime.')
    return json.loads(result.stdout.strip() or '{}')


def check_python_runtime():
    if not PYTHON_BIN.exists():
        raise RuntimeError('Virtual environment python missing; run initial install first.')
    version = subprocess.run([str(PYTHON_BIN), '--version'], capture_output=True, text=True)
    log(version.stdout.strip() or 'python --version produced no output.')
    try:
        info = _python_runtime_probe()
    except Exception as exc:
        log(f'Runtime probe failed: {exc}')
        return
    for name in ('wheel', 'setuptools', 'torch'):
        data = info.get(name, {})
        if not data.get('available'):
            log(f'{name}: not importable ({data.get("error", "unknown reason")}).')
        else:
            version_text = data.get('version') or 'version unknown'
            suffix = ''
            if name == 'torch':
                suffix = ' - CUDA ready' if data.get('cuda_available') else ' - CUDA unavailable'
            log(f'{name}: present ({version_text}){suffix}')


def repair_python_runtime():
    update_pip_tools()
    log('Ensuring wheel and setuptools packages are installed...')
    run_command([str(PYTHON_BIN), '-m', 'pip', 'install', '--upgrade', 'setuptools', 'wheel'])
    log('Python runtime tooling refreshed.')


def collect_config_report():
    details = []
    missing = []
    incorrect = []
    empty_files = []
    for entry in CONFIG_EXPECTATIONS:
        path = entry['path']
        expected_type = entry.get('type', 'file')
        optional = entry.get('optional', False)
        status = 'ok'
        severity = 'info' if optional else 'warn'
        exists = path.exists()
        real_path = None
        if exists:
            try:
                real_path = path.resolve()
            except OSError:
                real_path = None
        if not exists:
            status = 'missing'
            missing.append(str(path))
        elif expected_type == 'file' and not path.is_file():
            status = 'incorrect_type'
            incorrect.append(str(path))
        elif expected_type == 'dir' and not path.is_dir():
            status = 'incorrect_type'
            incorrect.append(str(path))
        elif path.is_file() and path.stat().st_size == 0:
            status = 'empty'
            empty_files.append(str(path))
        details.append({
            'path': str(path),
            'real_path': str(real_path) if real_path else None,
            'status': status,
            'expected': expected_type,
            'optional': optional,
            'severity': severity,
            'description': entry.get('description'),
        })
    summary = {
        'label': 'Configuration files',
        'missing': missing,
        'incorrect_type': incorrect,
        'items': details,
    }
    if empty_files:
        summary['empty'] = empty_files
    return summary


def check_configs():
    report = collect_config_report()
    log_package_findings({'configs': report})
    return report


def repair_configs():
    ensure_directories()
    created = 0
    for path, content in CONFIG_PLACEHOLDERS.items():
        if not path.parent.exists():
            path.parent.mkdir(parents=True, exist_ok=True)
        if not path.exists():
            path.write_text(content)
            log(f'Created placeholder config: {path}')
            created += 1
    if created == 0:
        log('No placeholder configs were needed; all tracked configs already exist.')
    report = collect_config_report()
    log_package_findings({'configs': report})


def check_cloudflared_dependencies():
    ensure_directories()
    required_tools = ['wget', 'dpkg-query']
    missing_tools = [tool for tool in required_tools if shutil.which(tool) is None]
    if missing_tools:
        log('Missing system utilities: ' + ', '.join(missing_tools))
    else:
        log('Required system utilities located: ' + ', '.join(required_tools))
    if CLOUDFLARED_BIN.exists():
        if os.access(CLOUDFLARED_BIN, os.X_OK):
            log('cloudflared binary present and executable.')
        else:
            log('cloudflared binary present but not executable; fixing permissions now.')
            os.chmod(CLOUDFLARED_BIN, 0o755)
    else:
        log('cloudflared binary not found under bin/.')
    if shutil.which('dpkg-query'):
        result = subprocess.run(['dpkg-query', '-W', '-f=${Status}', 'cloudflared'], capture_output=True, text=True)
        if result.returncode == 0 and 'installed' in result.stdout:
            log('An apt-managed cloudflared package is installed.')
        else:
            log('No apt-managed cloudflared package detected (standalone binary expected).')
    else:
        log('dpkg-query not available; skipping package database check.')


def install_cloudflared():
    ensure_directories()
    if CLOUDFLARED_BIN.exists():
        log('cloudflared already installed; use Update Cloudflared to refresh the binary.')
        return
    check_cloudflared_dependencies()
    download_cloudflared(force=True)


def update_cloudflared():
    ensure_directories()
    if CLOUDFLARED_BIN.exists():
        log('Removing existing cloudflared binary before refresh...')
        CLOUDFLARED_BIN.unlink(missing_ok=True)
    check_cloudflared_dependencies()
    download_cloudflared(force=True)


def compute_file_hash(path, block_size=65536, max_bytes=16 * 1024 * 1024):
    hasher = hashlib.sha256()
    read = 0
    with open(path, 'rb') as handle:
        while True:
            chunk = handle.read(block_size)
            if not chunk:
                break
            read += len(chunk)
            hasher.update(chunk)
            if max_bytes and read >= max_bytes:
                break
    return hasher.hexdigest(), read


def directory_signature(path, limit=FILES_SNAPSHOT_LIMIT):
    entries = []
    for root, _, files in os.walk(path):
        rel_root = os.path.relpath(root, path)
        for name in sorted(files):
            full_path = Path(root) / name
            try:
                stat = full_path.stat()
            except OSError:
                continue
            rel_path = os.path.normpath(os.path.join(rel_root, name))
            entries.append(f'{rel_path}:{stat.st_size}:{int(stat.st_mtime)}')
            if len(entries) >= limit:
                break
        if len(entries) >= limit:
            break
    digest = hashlib.sha256('|'.join(entries).encode('utf-8')).hexdigest() if entries else None
    return {'fingerprint': digest, 'sampled': len(entries), 'truncated': len(entries) >= limit}


def evaluate_target(key, meta):
    path = meta['path']
    exists = path.exists()
    entry = {
        'key': key,
        'path': str(path),
        'real_path': str(path.resolve()) if exists else None,
        'type': meta['type'],
        'description': meta['description'],
        'exists': exists,
    }
    if exists:
        try:
            stat = path.stat()
            entry['mtime'] = stat.st_mtime
        except OSError:
            entry['mtime'] = None
        if meta['type'] == 'file':
            digest, read = compute_file_hash(path)
            entry['signature'] = digest
            entry['bytes_sampled'] = read
        elif meta['type'] == 'dir':
            dir_info = directory_signature(path)
            entry.update(dir_info)
            sentinel = meta.get('sentinel')
            if sentinel is not None:
                entry['sentinel_exists'] = sentinel.exists()
            expected_files = meta.get('expected_files', [])
            missing_expected = []
            for rel_path in expected_files:
                expected_path = path / rel_path
                if not expected_path.exists():
                    missing_expected.append(rel_path)
            if missing_expected:
                entry['missing_expected_files'] = missing_expected
    else:
        entry['missing_since'] = time.time()
    return entry


def load_custom_integrity_paths():
    if not CUSTOM_PATHS_FILE.exists():
        return []
    try:
        data = json.loads(CUSTOM_PATHS_FILE.read_text())
        if isinstance(data, list):
            return data
        log('Custom integrity paths file is not a list; ignoring its contents.')
    except json.JSONDecodeError:
        log('Custom integrity paths file is not valid JSON; ignoring its contents.')
    return []


def merge_custom_targets(entries, custom_paths):
    for item in custom_paths:
        raw_path = item.get('path')
        if not raw_path:
            continue
        key = item.get('key') or f"custom::{Path(raw_path).name}"
        meta = {
            'path': Path(raw_path),
            'type': item.get('type', 'file'),
            'description': item.get('description', 'Custom path'),
        }
        if 'expected_files' in item:
            meta['expected_files'] = item['expected_files']
        entries[key] = evaluate_target(key, meta)


def capture_package_inventory():
    report = {}
    for key, spec in PACKAGE_EXPECTATIONS.items():
        missing = []
        incorrect_type = []
        outside_workspace = []
        items = []
        for item in spec['items']:
            path = item['path']
            expected_type = item.get('type', 'file')
            exists = path.exists()
            status = 'ok'
            real_path = None
            if exists:
                try:
                    real_path = path.resolve()
                except OSError:
                    real_path = None
            if not exists:
                missing.append(str(path))
                status = 'missing'
            elif expected_type == 'file' and not path.is_file():
                incorrect_type.append(str(path))
                status = 'incorrect_type'
            elif expected_type == 'dir' and not path.is_dir():
                incorrect_type.append(str(path))
                status = 'incorrect_type'
            if real_path and BASE_DIR not in real_path.parents and real_path != BASE_DIR:
                outside_workspace.append(str(real_path))
            items.append({'path': str(path), 'status': status, 'real_path': str(real_path) if real_path else None})
        report[key] = {
            'label': spec['label'],
            'missing': missing,
            'incorrect_type': incorrect_type,
            'outside_workspace': outside_workspace,
            'items': items,
        }
    report['configs'] = collect_config_report()
    return report


def gather_integrity_snapshot(include_packages=False):
    captured = time.time()
    entries = {key: evaluate_target(key, meta) for key, meta in INTEGRITY_TARGETS.items()}
    custom_paths = load_custom_integrity_paths()
    if custom_paths:
        merge_custom_targets(entries, custom_paths)
    snapshot = {'captured_at': captured, 'entries': entries, 'custom_paths': custom_paths}
    if include_packages:
        snapshot['package_inventory'] = capture_package_inventory()
    return snapshot


def load_integrity_state():
    state = {
        'baseline': None,
        'custom_paths': load_custom_integrity_paths(),
        'package_inventory': {},
        'component_status': {},
    }
    if INTEGRITY_STATE_FILE.exists():
        try:
            loaded = json.loads(INTEGRITY_STATE_FILE.read_text())
            if isinstance(loaded, dict):
                state.update(loaded)
        except json.JSONDecodeError:
            log('Integrity state file is not readable; starting fresh.')
    return state


def save_integrity_state(state):
    snapshot = dict(state)
    if 'custom_paths' not in snapshot:
        snapshot['custom_paths'] = load_custom_integrity_paths()
    INTEGRITY_STATE_FILE.parent.mkdir(parents=True, exist_ok=True)
    INTEGRITY_STATE_FILE.write_text(json.dumps(snapshot, indent=2))


def diff_snapshots(baseline, current):
    diff = {'missing': [], 'restored': [], 'changed': [], 'new': []}
    base_entries = baseline.get('entries', {})
    cur_entries = current.get('entries', {})
    for key, base in base_entries.items():
        cur = cur_entries.get(key)
        if not base.get('exists'):
            if cur and cur.get('exists'):
                diff['restored'].append(key)
            continue
        if not cur or not cur.get('exists'):
            diff['missing'].append(key)
            continue
        if base.get('signature') and cur.get('signature') and base['signature'] != cur['signature']:
            diff['changed'].append(key)
        elif base.get('fingerprint') and cur.get('fingerprint') and base['fingerprint'] != cur['fingerprint']:
            diff['changed'].append(key)
    for key, cur in cur_entries.items():
        if key not in base_entries and cur.get('exists'):
            diff['new'].append(key)
    return diff


def report_diff(diff):
    if diff['missing']:
        log('Missing targets: ' + ', '.join(diff['missing']))
    else:
        log('No targets missing compared to baseline.')
    if diff['changed']:
        log('Changed targets: ' + ', '.join(diff['changed']))
    if diff['restored']:
        log('Restored targets since baseline: ' + ', '.join(diff['restored']))


def log_package_findings(inventory):
    for key, data in inventory.items():
        label = data.get('label', key)
        missing = data.get('missing', [])
        incorrect = data.get('incorrect_type', [])
        outside = data.get('outside_workspace', [])
        empty = data.get('empty', [])
        issues = []
        if missing:
            issues.append(f'missing {len(missing)}')
        if incorrect:
            issues.append(f'type mismatches {len(incorrect)}')
        if outside:
            issues.append(f'outside workspace {len(outside)}')
        if empty:
            issues.append(f'empty files {len(empty)}')
        if issues:
            log(f'{label}: issues detected ({"; ".join(issues)}).')
            for path in missing:
                log(f'  - missing: {path}')
            for path in incorrect:
                log(f'  - incorrect type: {path}')
            for path in outside:
                log(f'  - outside workspace: {path}')
            for path in empty:
                log(f'  - empty file: {path}')
        else:
            log(f'{label}: all expected files present.')


def apply_component_status(state, **updates):
    component_status = state.setdefault('component_status', {})
    now = time.time()
    for key, value in updates.items():
        component_status[key] = {'status': value, 'timestamp': now}
    return state


def check_integrity():
    ensure_directories()
    state = load_integrity_state()
    snapshot = gather_integrity_snapshot(include_packages=True)
    baseline = state.get('baseline')
    if not baseline:
        state['baseline'] = snapshot
        diff = {'missing': [], 'restored': [], 'changed': [], 'new': []}
        log('Integrity baseline created. Run future scans to detect drift.')
    else:
        diff = diff_snapshots(baseline, snapshot)
        report_diff(diff)
    inventory = snapshot.get('package_inventory', {})
    if inventory:
        log_package_findings(inventory)
    state['last_scan'] = snapshot
    state['last_diff'] = diff
    state['package_inventory'] = inventory
    save_integrity_state(state)
    refresh_integrity_summary(state)
    return state


def scan_installation():
    ensure_directories()
    inventory = capture_package_inventory()
    log_package_findings(inventory)
    state = load_integrity_state()
    state['package_inventory'] = inventory
    save_integrity_state(state)
    refresh_integrity_summary(state)
    return inventory


def augment_integrity_baseline():
    ensure_directories()
    snapshot = gather_integrity_snapshot(include_packages=True)
    state = load_integrity_state()
    state['baseline'] = snapshot
    state['last_scan'] = snapshot
    state['last_diff'] = {'missing': [], 'restored': [], 'changed': [], 'new': []}
    state['package_inventory'] = snapshot.get('package_inventory', {})
    save_integrity_state(state)
    log('Integrity baseline replaced with current snapshot (including package inventory and custom paths).')
    refresh_integrity_summary(state)
    return state


def repair_manager_repo():
    ensure_manager_repo(update=True)


def repair_missing():
    ensure_directories()
    state = load_integrity_state()
    baseline = state.get('baseline')
    if not baseline:
        log('No baseline recorded yet; run Check Integrity first.')
        return state
    current = gather_integrity_snapshot(include_packages=True)
    diff = diff_snapshots(baseline, current)
    missing = diff['missing']
    if not missing:
        log('No missing targets detected. Nothing to repair.')
        state['last_scan'] = current
        state['last_diff'] = diff
        state['package_inventory'] = current.get('package_inventory', {})
        save_integrity_state(state)
        refresh_integrity_summary(state)
        return state
    for key in missing:
        handler = REPAIR_HANDLERS.get(key)
        meta = INTEGRITY_TARGETS.get(key, {})
        label = meta.get('description', key)
        if handler:
            log(f'Attempting repair for {label}...')
            try:
                handler()
            except Exception as exc:
                log(f'Repair for {key} failed: {exc}')
        else:
            log(f'No automated repair available for {label}.')
    refreshed = check_integrity()
    return refreshed


def rehydrate_environment():
    ensure_directories()
    if not COMFY_DIR.exists():
        log('ComfyUI directory not found; rehydration requires an existing workspace copy.')
        return None
    status_updates = {}
    if not VENV_DIR.exists():
        log('Virtual environment missing; creating a fresh copy in persistent storage.')
        create_virtualenv()
        status_updates['virtualenv'] = 'created'
    else:
        log('Virtual environment present; reusing existing interpreter.')
        status_updates['virtualenv'] = 'existing'
    update_pip_tools()
    status_updates['pip_tools'] = 'updated'
    install_dependencies()
    status_updates['dependencies'] = 'refreshed'
    ensure_manager_repo(update=True)
    refresh_custom_nodes()
    status_updates['custom_nodes'] = 'hydrated'
    check_cloudflared_dependencies()
    download_cloudflared(force=False)
    status_updates['cloudflared'] = 'verified'
    check_configs()
    status_updates['configs'] = 'audited'
    check_python_runtime()
    status_updates['python_runtime'] = 'verified'
    scan_installation()
    state = load_integrity_state()
    state = apply_component_status(state, **status_updates)
    state['rehydrated_at'] = time.time()
    save_integrity_state(state)
    refreshed = check_integrity()
    log('Rehydration sequence finished. Consider updating the integrity baseline once satisfied.')
    return refreshed


REPAIR_HANDLERS = {
    'comfy_repo': ensure_comfy_repo,
    'python_binary': create_virtualenv,
    'pip_binary': create_virtualenv,
    'cloudflared': install_cloudflared,
    'manager_repo': repair_manager_repo,
}


ACTIONS = {
    'initial_install': {
        'label': 'Initial install',
        'func': initial_install,
        'help': 'Clone ComfyUI, set up the virtual env, download cloudflared, and install baseline dependencies.',
    },
    'install_dependencies': {
        'label': 'Install dependencies',
        'func': install_dependencies,
        'help': 'Reinstall the base, manager, and custom node Python requirements.',
    },
    'check_dependencies': {
        'label': 'Check dependencies',
        'func': check_dependencies,
        'help': 'Run pip check to identify conflicts and report issues.',
    },
    'update_comfyui': {
        'label': 'Update ComfyUI',
        'func': update_comfyui,
        'help': 'Pull the latest changes from GitHub into the persistent checkout.',
    },
    'update_manager': {
        'label': 'Refresh ComfyUI Manager',
        'func': update_manager,
        'help': 'Clone or update the ComfyUI Manager custom node and install its requirements.',
    },
    'update_custom_nodes': {
        'label': 'Refresh custom nodes',
        'func': update_custom_nodes,
        'help': 'Install requirements.txt for each custom node under custom_nodes.',
    },
    'check_pip_tools': {
        'label': 'Check pip status',
        'func': check_pip_tools,
        'help': 'Inspect pip version and list outdated packages in the virtual environment.',
    },
    'update_pip_tools': {
        'label': 'Update pip tooling',
        'func': update_pip_tools,
        'help': 'Upgrade pip, setuptools, wheel, and packaging to the latest versions.',
    },
    'check_python_runtime': {
        'label': 'Check python runtime',
        'func': check_python_runtime,
        'help': 'Probe python, wheel/setuptools, and torch CUDA availability.',
    },
    'repair_python_runtime': {
        'label': 'Repair python runtime',
        'func': repair_python_runtime,
        'help': 'Reinstall wheel and setuptools after upgrading core packaging tools.',
    },
    'check_configs': {
        'label': 'Check configs',
        'func': check_configs,
        'help': 'Audit configuration files and directories for presence and type.',
    },
    'repair_configs': {
        'label': 'Repair configs',
        'func': repair_configs,
        'help': 'Create placeholder configs and directories if any are missing.',
    },
    'check_cloudflared_dependencies': {
        'label': 'Check cloudflared dependencies',
        'func': check_cloudflared_dependencies,
        'help': 'Verify required system binaries, dpkg metadata, and binary permissions.',
    },
    'install_cloudflared': {
        'label': 'Install cloudflared',
        'func': install_cloudflared,
        'help': 'Download the cloudflared binary into persistent storage if missing.',
    },
    'update_cloudflared': {
        'label': 'Update cloudflared',
        'func': update_cloudflared,
        'help': 'Redownload the latest cloudflared binary and refresh permissions.',
    },
    'scan_installation': {
        'label': 'Scan installation paths',
        'func': scan_installation,
        'help': 'Validate expected files per package and surface missing or misplaced assets.',
    },
    'check_integrity': {
        'label': 'Check integrity',
        'func': check_integrity,
        'help': 'Run a read-only scan of key assets and compare against the saved baseline.',
    },
    'repair_missing': {
        'label': 'Repair missing items',
        'func': repair_missing,
        'help': 'Attempt to restore any baseline targets that are currently missing.',
    },
    'augment_baseline': {
        'label': 'Update integrity baseline',
        'func': augment_integrity_baseline,
        'help': 'Replace the stored baseline with the current state, package inventory, and custom paths.',
    },
    'rehydrate_environment': {
        'label': 'Rehydrate environment',
        'func': rehydrate_environment,
        'help': 'Restore a copied workspace by rebuilding the virtual env, tooling, configs, and verifying state.',
    },
}


def format_timestamp(ts):
    if not ts:
        return 'never'
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ts))


def summarize_package_inventory(inventory):
    if not inventory:
        return 'No package scan recorded yet.'
    issues = []
    for data in inventory.values():
        missing = len(data.get('missing', []))
        incorrect = len(data.get('incorrect_type', []))
        outside = len(data.get('outside_workspace', []))
        empty = len(data.get('empty', [])) if isinstance(data.get('empty'), list) else 0
        if missing or incorrect or outside or empty:
            parts = []
            if missing:
                parts.append(f'missing {missing}')
            if incorrect:
                parts.append(f'type issues {incorrect}')
            if outside:
                parts.append(f'outside workspace {outside}')
            if empty:
                parts.append(f'empty files {empty}')
            issues.append(f"{data.get('label', 'package')}: {'; '.join(parts)}")
    if not issues:
        return 'All tracked packages healthy.'
    return '; '.join(issues)


def summarize_component_status(component_status):
    if not component_status:
        return 'No maintenance status recorded.'
    lines = []
    for key in sorted(component_status):
        info = component_status[key] or {}
        status = info.get('status', 'unknown')
        timestamp = format_timestamp(info.get('timestamp'))
        label = key.replace('_', ' ').title()
        lines.append(f'{label}: {status} (at {timestamp})')
    return '; '.join(lines)


def refresh_integrity_summary(state=None):
    if state is None:
        state = load_integrity_state()
    baseline = state.get('baseline') or {}
    last_scan = state.get('last_scan') or {}
    diff = state.get('last_diff') or {'missing': [], 'restored': [], 'changed': [], 'new': []}
    baseline_time = format_timestamp(baseline.get('captured_at'))
    last_time = format_timestamp(last_scan.get('captured_at'))
    missing = ', '.join(diff.get('missing', [])) or 'none'
    changed = ', '.join(diff.get('changed', [])) or 'none'
    restored = ', '.join(diff.get('restored', [])) or 'none'
    inventory_summary = summarize_package_inventory(state.get('package_inventory', {}))
    custom_paths_count = len(state.get('custom_paths') or [])
    component_summary = summarize_component_status(state.get('component_status', {}))
    summary_html.value = (
        "<div style='background:#0b1120;border:1px solid #1f2937;border-radius:12px;padding:12px;color:#e2e8f0;'>"
        "<div style='font-size:14px;font-weight:600;'>Integrity summary</div>"
        f"<div style='margin-top:6px;font-size:12px;'>Baseline captured: {baseline_time}</div>"
        f"<div style='font-size:12px;'>Last scan: {last_time}</div>"
        f"<div style='margin-top:8px;font-size:12px;'>Missing targets: {missing}</div>"
        f"<div style='font-size:12px;'>Changed targets: {changed}</div>"
        f"<div style='font-size:12px;'>Recently restored: {restored}</div>"
        f"<div style='margin-top:8px;font-size:12px;'>Package scans: {inventory_summary}</div>"
        f"<div style='font-size:12px;'>Custom targets tracked: {custom_paths_count}</div>"
        f"<div style='margin-top:8px;font-size:12px;'>Component status: {component_summary}</div>"
        f"<div style='margin-top:8px;font-size:12px;color:#94a3b8;'>State file: {INTEGRITY_STATE_FILE}</div>"
        f"<div style='font-size:12px;color:#94a3b8;'>Custom paths file (optional): {CUSTOM_PATHS_FILE}</div>"
        "</div>"
    )


def on_select_all(_):
    for checkbox in step_checkboxes.values():
        checkbox.value = True


def on_select_none(_):
    for checkbox in step_checkboxes.values():
        checkbox.value = False


def execute_steps(step_ids):
    global ACTION_LOG_WIDGET
    ACTION_LOG_WIDGET = action_log
    action_log.clear_output()
    for step_id in step_ids:
        action = ACTIONS[step_id]
        label = action['label']
        log('')
        log(f'=== {label} ===')
        started = time.time()
        try:
            action['func']()
            elapsed = time.time() - started
            log(f'{label} completed in {elapsed:.1f} seconds.')
        except Exception as exc:
            log(f'{label} failed: {exc}')
            break
    ACTION_LOG_WIDGET = None
    refresh_integrity_summary()


def on_run_clicked(_):
    chosen = [key for key, checkbox in step_checkboxes.items() if checkbox.value]
    if not chosen:
        with action_log:
            action_log.clear_output()
            print('Select at least one action to run.')
        return
    run_button.disabled = True
    try:
        execute_steps(chosen)
    finally:
        run_button.disabled = False


step_checkboxes = {}
for key, action in ACTIONS.items():
    estimate = STEP_ESTIMATES.get(key, '')
    description = f"{action['label']} ({estimate})"
    checkbox = widgets.Checkbox(value=False, description=description, indent=False)
    step_checkboxes[key] = checkbox

run_button = widgets.Button(description='Run selected actions', button_style='primary', icon='play')
run_button.on_click(on_run_clicked)

select_all_button = widgets.Button(description='Select all', icon='check-square')
select_all_button.on_click(on_select_all)

select_none_button = widgets.Button(description='Clear selection', icon='square-o')
select_none_button.on_click(on_select_none)

action_log = widgets.Output(layout={'border': '1px solid #1f1f1f', 'padding': '10px', 'max_height': '260px', 'overflow_y': 'auto', 'background': '#050505', 'color': '#e5e7eb', 'font_family': 'Menlo,monospace'})
summary_html = widgets.HTML()

estimate_rows = ''.join(
    f"<tr><td style='padding:4px 8px;border:1px solid #1f2937;'>{ACTIONS[key]['label']}</td>"
    f"<td style='padding:4px 8px;border:1px solid #1f2937;'>{STEP_ESTIMATES.get(key, 'n/a')}</td>"
    f"<td style='padding:4px 8px;border:1px solid #1f2937;'>{ACTIONS[key]['help']}</td></tr>"
    for key in ACTIONS
)

estimates_table = widgets.HTML(
    value=(
        "<div style='margin-bottom:10px;'>"
        "<div style='font-size:14px;font-weight:600;color:#cbd5f5;'>Action guide</div>"
        "<table style='border-collapse:collapse;margin-top:6px;font-size:12px;color:#e2e8f0;'>"
        "<thead><tr>"
        "<th style='padding:4px 8px;border:1px solid #1f2937;'>Action</th>"
        "<th style='padding:4px 8px;border:1px solid #1f2937;'>Estimate</th>"
        "<th style='padding:4px 8px;border:1px solid #1f2937;'>What it does</th>"
        "</tr></thead>"
        f"<tbody>{estimate_rows}</tbody>"
        "</table>"
        "</div>"
    )
)

refresh_integrity_summary()

controls = widgets.VBox([
    estimates_table,
    summary_html,
    widgets.HBox([select_all_button, select_none_button, run_button], layout=widgets.Layout(gap='10px', flex_flow='row wrap')),
    widgets.VBox(list(step_checkboxes.values()), layout=widgets.Layout(margin='6px 0 0 0')),
    action_log,
])

display(controls)


`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/requirements.txt`

`$ git -C /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/ComfyUI-Manager pull --ff-only`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/ComfyUI-Manager/requirements.txt`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/ComfyUI-GGUF/requirements.txt`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/ComfyUI-Manager/requirements.txt`

...

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/comfyui-detail-daemon/requirements.txt`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/comfyui-impact-pack/requirements.txt`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/comfyui-impact-subpack/requirements.txt`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/comfyui_essentials/requirements.txt`

`$ /content/drive/MyDrive/comfyui_env/venv/bin/python -m pip install -r /content/drive/MyDrive/comfyui_env/ComfyUI/custom_nodes/rgthree-comfy/requirements.txt`

In [ ]:
import socket, sys
s=socket.socket()
s.settimeout(3)
try:
    s.connect(('127.0.0.1', 8188))
    print('connect ok')
except Exception as e:
    print('connect failed:', repr(e))
finally:
    s.close()


In [3]:
# Backup, restore, and targeted-readonly-check helpers
import json, zipfile, shutil, time, os
from pathlib import Path

# Backup configuration
BACKUP_DIR = BASE_DIR / 'backups'
BACKUP_CONFIG = RUNTIME_DIR / 'backup_config.json'
DEFAULT_BACKUP_CONFIG = {
    'local_backup_path': str(BACKUP_DIR),
    'github_repo_path': '',
    'keep': 8,
}
BACKUP_TIMESTAMP_FMT = '%Y%m%d-%H%M%S'


def ensure_backup_dirs():
    BACKUP_DIR.mkdir(parents=True, exist_ok=True)
    RUNTIME_DIR.mkdir(parents=True, exist_ok=True)


def load_backup_config():
    ensure_backup_dirs()
    if BACKUP_CONFIG.exists():
        try:
            cfg = json.loads(BACKUP_CONFIG.read_text())
            return {**DEFAULT_BACKUP_CONFIG, **cfg}
        except Exception:
            log('Backup config corrupted; reverting to defaults.')
    return dict(DEFAULT_BACKUP_CONFIG)


def save_backup_config(cfg):
    ensure_backup_dirs()
    BACKUP_CONFIG.write_text(json.dumps(cfg, indent=2))


def create_backup(include_paths=None, message=None):
    """Create a timestamped zip of the ComfyUI workspace (or provided paths).
    Returns path to archive."""
    ensure_backup_dirs()
    ts = time.strftime(BACKUP_TIMESTAMP_FMT)
    name = f'comfyui-backup-{ts}.zip'
    dest = BACKUP_DIR / name
    paths = include_paths or [str(COMFY_DIR)]
    with zipfile.ZipFile(dest, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        for base in paths:
            base = Path(base)
            if not base.exists():
                continue
            if base.is_file():
                zf.write(base, arcname=base.name)
            else:
                for root, _, files in os.walk(base):
                    for fname in files:
                        fp = Path(root) / fname
                        try:
                            arc = fp.relative_to(base.parent)
                        except Exception:
                            arc = fp.name
                        try:
                            zf.write(fp, arcname=str(arc))
                        except OSError:
                            continue
    log(f'Backup created: {dest}')
    # enforce retention
    cfg = load_backup_config()
    keep = int(cfg.get('keep', 8) or 8)
    archives = sorted(BACKUP_DIR.glob('comfyui-backup-*.zip'), key=lambda p: p.stat().st_mtime, reverse=True)
    for old in archives[keep:]:
        try:
            old.unlink()
        except Exception:
            pass
    return str(dest)


def list_backups():
    ensure_backup_dirs()
    archives = sorted(BACKUP_DIR.glob('comfyui-backup-*.zip'), key=lambda p: p.stat().st_mtime, reverse=True)
    return [{'path': str(a), 'mtime': a.stat().st_mtime, 'size': a.stat().st_size} for a in archives]


def restore_backup(archive_path, merge_missing=True):
    """Restore files from archive. By default only extracts files missing in target location."""
    archive = Path(archive_path)
    if not archive.exists():
        raise FileNotFoundError(archive_path)
    extracted = []
    skipped = []
    with zipfile.ZipFile(archive, 'r') as zf:
        for member in zf.namelist():
            target = BASE_DIR / member
            if target.exists() and merge_missing:
                skipped.append(member)
                continue
            target.parent.mkdir(parents=True, exist_ok=True)
            try:
                with zf.open(member) as src, open(target, 'wb') as out:
                    shutil.copyfileobj(src, out)
                extracted.append(member)
            except Exception as exc:
                log(f'Failed to extract {member}: {exc}')
    log(f'Restore complete. Extracted: {len(extracted)} files, skipped: {len(skipped)}')
    # After restore, refresh minimal environment wiring
    try:
        scan_installation()
    except Exception:
        pass
    return {'extracted': extracted, 'skipped': skipped}


def check_files_readonly():
    """Read-only snapshot report used for health checks and diagnostics."""
    ensure_directories()
    snapshot = gather_integrity_snapshot(include_packages=True)
    entries = snapshot.get('entries', {})
    report = {'missing': [], 'changed': [], 'new': []}
    for key, data in entries.items():
        if not data.get('exists'):
            report['missing'].append(key)
    return {'snapshot': snapshot, 'report': report}


def run_repair_missing(targets=None):
    """Attempt targeted repairs for provided integrity keys. Returns a summary."""
    ensure_directories()
    if targets is None:
        return repair_missing()
    repaired = []
    failed = []
    for key in targets:
        handler = REPAIR_HANDLERS.get(key)
        meta = INTEGRITY_TARGETS.get(key, {})
        label = meta.get('description', key)
        if handler:
            try:
                log(f'Attempting targeted repair for {label}...')
                handler()
                repaired.append(key)
            except Exception as exc:
                log(f'Targeted repair for {key} failed: {exc}')
                failed.append(key)
        else:
            log(f'No automated repair handler for {label}.')
            failed.append(key)
    refreshed = check_integrity()
    return {'repaired': repaired, 'failed': failed, 'refreshed': refreshed}


## 2. Launch Cloudflared & ComfyUI
Once dependencies are ready, run this cell to terminate any old background processes, download the Cloudflare tunnel if needed, and restart both services in nohup-style mode. Progress messages indicate each stage, followed by the public URL when available.

In [18]:
import json
import os
import re
import signal
import subprocess
import tempfile
import time
import zipfile
from pathlib import Path

from IPython.display import HTML, Markdown, display

# --- Shared paths and persistent assets ---
BASE_DIR = Path('/content/drive/MyDrive/comfyui_env')
COMFY_DIR = BASE_DIR / 'ComfyUI'
VENV_DIR = BASE_DIR / 'venv'
BIN_DIR = BASE_DIR / 'bin'
LOG_DIR = BASE_DIR / 'logs'
RUNTIME_DIR = BASE_DIR / 'runtime'
PROCESS_FILE = RUNTIME_DIR / 'processes.json'
URL_FILE = RUNTIME_DIR / 'last_url.txt'
CLOUDFLARED_PATH = BIN_DIR / 'cloudflared'
CLOUDFLARED_LOG = LOG_DIR / 'cloudflared.log'
COMFY_LOG = LOG_DIR / 'comfyui.log'
PYTHON_BIN = VENV_DIR / 'bin' / 'python'
SESSION_LOG = RUNTIME_DIR / 'session_history.log'

SESSION_LOG.parent.mkdir(parents=True, exist_ok=True)
if not SESSION_LOG.exists():
    SESSION_LOG.write_text('=== ComfyUI Session Log ===\n')

def append_history(message):
    """Write a timestamped message to the shared session log."""
    timestamp = time.strftime('%H:%M:%S')
    line = f'[{timestamp}] {message}'
    with open(SESSION_LOG, 'a', encoding='utf-8') as fh:
        fh.write(line + '\n')
    return line

def log_status(message):
    """Surface launch events in output while persisting them."""
    line = append_history(message)
    print(line)

def load_process_state():
    if PROCESS_FILE.exists():
        try:
            return json.loads(PROCESS_FILE.read_text())
        except json.JSONDecodeError:
            log_status('Warning: process metadata corrupt; starting fresh.')
    return {}

def save_process_state(state):
    PROCESS_FILE.parent.mkdir(parents=True, exist_ok=True)
    PROCESS_FILE.write_text(json.dumps(state, indent=2))

def run_command(command, cwd=None):
    display(Markdown("`$ {}`".format(' '.join(command))))
    append_history(f"$ {' '.join(command)}")
    subprocess.run(command, cwd=str(cwd) if cwd else None, check=True)

def ensure_cloudflared():
    """Persistently download cloudflared if it is not present and enforce executable bit."""
    log_status('Ensuring cloudflared binary is available...')
    BIN_DIR.mkdir(parents=True, exist_ok=True)
    if CLOUDFLARED_PATH.exists():
        log_status('cloudflared already present in persistent storage; refreshing permissions.')
        os.chmod(CLOUDFLARED_PATH, 0o755)
        return
    log_status('Downloading cloudflared (first-time setup may take ~1 minute)...')
    url = 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64'
    run_command(['wget', '-q', '-O', str(CLOUDFLARED_PATH), url])
    os.chmod(CLOUDFLARED_PATH, 0o755)
    log_status('cloudflared download finished.')

def stop_process(name, state):
    entry = state.get(name)
    if not entry:
        return
    pid = entry.get('pid')
    if pid:
        log_status(f'Stopping lingering {name} process (pid {pid})...')
        try:
            os.killpg(pid, signal.SIGTERM)
        except ProcessLookupError:
            pass
        except OSError:
            try:
                os.kill(pid, signal.SIGTERM)
            except OSError:
                pass
        time.sleep(1)
    state.pop(name, None)

log_status('Loading previous process metadata...')
state = load_process_state()
stop_process('cloudflared', state)
stop_process('comfyui', state)
save_process_state(state)
log_status('Previous background processes stopped (if any).')

ensure_cloudflared()

log_status('Resetting log files for fresh run...')
for log_path in (CLOUDFLARED_LOG, COMFY_LOG):
    log_path.unlink(missing_ok=True)

if not PYTHON_BIN.exists():
    raise FileNotFoundError('Virtual environment missing — run Cell 1 first.')
if not COMFY_DIR.exists():
    raise FileNotFoundError('ComfyUI directory missing — run Cell 1 initial install.')

log_status('Preparing environment variables for background services...')
env = os.environ.copy()
env['PATH'] = f"{VENV_DIR / 'bin'}:{BIN_DIR}:{env.get('PATH', '')}"
env['VIRTUAL_ENV'] = str(VENV_DIR)
env['PYTHONUNBUFFERED'] = '1'

cloudflared_cmd = [str(CLOUDFLARED_PATH), 'tunnel', '--no-autoupdate', '--url', 'http://127.0.0.1:8188']
comfy_cmd = [str(PYTHON_BIN), str(COMFY_DIR / 'main.py'), '--listen', '0.0.0.0', '--port', '8188']

log_status('Launching cloudflared tunnel process...')
cloudflared_log_handle = open(CLOUDFLARED_LOG, 'ab')
# Use setsid so the process behaves like nohup: it will keep running after the cell exits.
try:
    cloudflared_proc = subprocess.Popen(
        cloudflared_cmd,
        stdout=cloudflared_log_handle,
        stderr=subprocess.STDOUT,
        cwd=str(BASE_DIR),
        env=env,
        preexec_fn=os.setsid,
    )
except PermissionError:
    log_status('cloudflared was not executable; reapplying chmod and retrying...')
    os.chmod(CLOUDFLARED_PATH, 0o755)
    cloudflared_proc = subprocess.Popen(
        cloudflared_cmd,
        stdout=cloudflared_log_handle,
        stderr=subprocess.STDOUT,
        cwd=str(BASE_DIR),
        env=env,
        preexec_fn=os.setsid,
    )
cloud_started = time.time()
log_status(f'cloudflared started with pid {cloudflared_proc.pid}. Logs streaming to {CLOUDFLARED_LOG}.')

log_status('Launching ComfyUI server process...')
comfy_log_handle = open(COMFY_LOG, 'ab')
# Same nohup-style detach for the ComfyUI server process.
comfy_proc = subprocess.Popen(
    comfy_cmd,
    stdout=comfy_log_handle,
    stderr=subprocess.STDOUT,
    cwd=str(COMFY_DIR),
    env=env,
    preexec_fn=os.setsid,
)
comfy_started = time.time()
log_status(f'ComfyUI server started with pid {comfy_proc.pid}. Logs streaming to {COMFY_LOG}.')

state = load_process_state()
state['cloudflared'] = {
    'pid': cloudflared_proc.pid,
    'cmd': cloudflared_cmd,
    'cwd': str(BASE_DIR),
    'log': str(CLOUDFLARED_LOG),
    'env': {'PATH': env['PATH'], 'VIRTUAL_ENV': env['VIRTUAL_ENV'], 'PYTHONUNBUFFERED': env['PYTHONUNBUFFERED']},
    'started_at': cloud_started,
}
state['comfyui'] = {
    'pid': comfy_proc.pid,
    'cmd': comfy_cmd,
    'cwd': str(COMFY_DIR),
    'log': str(COMFY_LOG),
    'env': {'PATH': env['PATH'], 'VIRTUAL_ENV': env['VIRTUAL_ENV'], 'PYTHONUNBUFFERED': env['PYTHONUNBUFFERED']},
    'started_at': comfy_started,
}
save_process_state(state)
log_status('Process metadata saved for control panel.')

log_status('Waiting for Cloudflared to provide a public URL (timeout ~3 minutes)...')
url_pattern = re.compile(r'https://[-\w.]+\.trycloudflare\.com')
public_url = None
for attempt in range(120):
    if CLOUDFLARED_LOG.exists():
        log_text = CLOUDFLARED_LOG.read_text(errors='ignore')
        match = url_pattern.search(log_text)
        if match:
            public_url = match.group(0)
            break
    if attempt % 15 == 0 and attempt > 0:
        log_status('Still waiting on tunnel URL...')
    time.sleep(1.5)

if public_url:
    os.environ['COMFY_URL'] = public_url
    URL_FILE.write_text(public_url)
    display(HTML(f"<div style='margin:6px 0;'><a href='{public_url}' target='_blank' style='font-weight:600;color:#2563eb;text-decoration:none;'>Open ComfyUI (Cloudflared)</a></div>"))
    log_status('Tunnel is live; URL stored for control panel.')
else:
    log_status('Timed out waiting for the Cloudflared URL. Check cloudflared.log for details.')

display(Markdown('- cloudflared running in the background (nohup-style via setsid).'))
display(Markdown('- ComfyUI server running in the background; logs in comfyui.log.'))
display(Markdown('- Proceed to the control panel cell for monitoring and restarts.'))
log_status('Startup sequence finished. Move to the control panel to monitor services.')

[00:27:56] Loading previous process metadata...
[00:27:56] Stopping lingering cloudflared process (pid 2104)...
[00:27:57] Stopping lingering comfyui process (pid 2114)...
[00:27:58] Previous background processes stopped (if any).
[00:27:58] Ensuring cloudflared binary is available...
[00:27:58] cloudflared already present in persistent storage; refreshing permissions.
[00:27:58] Resetting log files for fresh run...
[00:27:58] Preparing environment variables for background services...
[00:27:58] Launching cloudflared tunnel process...
[00:27:58] cloudflared started with pid 8196. Logs streaming to /content/drive/MyDrive/comfyui_env/logs/cloudflared.log.
[00:27:58] Launching ComfyUI server process...
[00:27:58] ComfyUI server started with pid 8197. Logs streaming to /content/drive/MyDrive/comfyui_env/logs/comfyui.log.
[00:27:58] Process metadata saved for control panel.
[00:27:58] Waiting for Cloudflared to provide a public URL (timeout ~3 minutes)...


[00:28:04] Tunnel is live; URL stored for control panel.


- cloudflared running in the background (nohup-style via setsid).

- ComfyUI server running in the background; logs in comfyui.log.

- Proceed to the control panel cell for monitoring and restarts.

[00:28:04] Startup sequence finished. Move to the control panel to monitor services.


In [ ]:
# Normalize saved tunnel URL by appending ComfyUI port (:8188) when needed
from pathlib import Path
from urllib.parse import urlparse, urlunparse

RUNTIME_DIR = Path('/content/drive/MyDrive/comfyui_env/runtime')
URL_FILE = RUNTIME_DIR / 'last_url.txt'


def ensure_url_with_port(port=8188):
    if not URL_FILE.exists():
        print('No saved tunnel URL to normalize.')
        return None
    url = URL_FILE.read_text().strip()
    if not url:
        print('Saved URL is empty.')
        return None
    p = urlparse(url)
    if p.port is None:
        # Build netloc with explicit port
        host = p.hostname or ''
        if not host:
            print('Could not parse hostname from URL:', url)
            return url
        netloc = host + f':{port}'
        if p.username:
            netloc = f'{p.username}@' + netloc
        new = urlunparse((p.scheme, netloc, p.path or '', p.params or '', p.query or '', p.fragment or ''))
        URL_FILE.write_text(new)
        os.environ['COMFY_URL'] = new
        print('Updated tunnel URL to include port:', new)
        return new
    else:
        print('Tunnel URL already includes a port:', url)
        return url

# Run normalization immediately (idempotent)
try:
    ensure_url_with_port(8188)
except Exception as e:
    print('Port normalization failed:', e)


## 3. Monitor, Restart, and Tail Logs
Use the widget panel to check service status, stream logs, and perform targeted restarts. The buttons in this section operate on the metadata captured during the launch step.

In [19]:
import json

import os

import re

import signal

import subprocess

import threading

import time

from html import escape

from pathlib import Path



import ipywidgets as widgets

from IPython.display import display



try:

    import requests

except ImportError:

    requests = None



# Gracefully tear down previously created widgets/threads if the cell is rerun.

if 'tail_stop_event' in globals():

    try:

        tail_stop_event.set()

    except Exception:

        pass

    existing_tail = globals().get('tail_thread')

    if existing_tail and existing_tail.is_alive():

        existing_tail.join(timeout=0.3)



if 'auto_refresh_stop_event' in globals():

    try:

        auto_refresh_stop_event.set()

    except Exception:

        pass

    existing_auto = globals().get('auto_refresh_thread')

    if existing_auto and existing_auto.is_alive():

        existing_auto.join(timeout=0.3)



# --- Persistent runtime references ---

BASE_DIR = Path('/content/drive/MyDrive/comfyui_env')

LOG_DIR = BASE_DIR / 'logs'

RUNTIME_DIR = BASE_DIR / 'runtime'

PROCESS_FILE = RUNTIME_DIR / 'processes.json'

URL_FILE = RUNTIME_DIR / 'last_url.txt'

CLOUDFLARED_LOG = LOG_DIR / 'cloudflared.log'

COMFY_LOG = LOG_DIR / 'comfyui.log'

SESSION_LOG = RUNTIME_DIR / 'session_history.log'



SESSION_LOG.parent.mkdir(parents=True, exist_ok=True)

if not SESSION_LOG.exists():

    SESSION_LOG.write_text('=== ComfyUI Session Log ===\n')



tail_stop_event = threading.Event()

tail_thread = None



auto_refresh_stop_event = threading.Event()

auto_refresh_thread = None



AUTO_REFRESH_INTERVAL = 6  # seconds between automatic dashboard refreshes

REMOTE_CHECK_INTERVAL = 30  # seconds between remote tunnel probes



dashboard_last_update = {'timestamp': None}



health_snapshot = {

    'cloudflared': {

        'running': False,

        'uptime': '—',

        'pid': None,

        'remote_ok': None,

        'remote_latency': None,

        'remote_status_code': None,

        'remote_error': None,

        'remote_checked_at': None,

        'last_checked': None,

    },

    'comfyui': {

        'running': False,

        'uptime': '—',

        'pid': None,

        'local_ok': None,

        'local_latency': None,

        'local_status_code': None,

        'local_error': None,

        'remote_ok': None,

        'remote_latency': None,

        'remote_status_code': None,

        'remote_error': None,

        'remote_checked_at': None,

        'last_checked': None,

    },

}



def abbreviate(text, limit=80):

    if text is None:

        return None

    text = str(text)

    return text if len(text) <= limit else text[:limit - 3] + '...'



def append_history(message):

    timestamp = time.strftime('%H:%M:%S')

    line = f'[{timestamp}] {message}'

    with open(SESSION_LOG, 'a', encoding='utf-8') as fh:

        fh.write(line + '\n')

    return line



def load_history_html(limit_bytes=16000):

    if not SESSION_LOG.exists():

        return (

            "<div style='background:#080808;border:1px solid #1f1f1f;border-radius:10px;padding:12px;color:#9ca3af;font-family:Menlo,monospace;'>"

            "No session activity recorded yet."

            "</div>"

        )

    text = SESSION_LOG.read_text(errors='ignore')

    if len(text) > limit_bytes:

        text = '... previous entries truncated ...\n' + text[-limit_bytes:]

    return (

        "<div style='background:#080808;border:1px solid #1f1f1f;border-radius:10px;padding:12px;font-family:Menlo,monospace;color:#d0d6e1;line-height:1.4;'>"

        "<div style='font-size:12px;letter-spacing:0.1em;color:#7aa2f7;'>SESSION TIMELINE</div>"

        f"<pre style='margin:8px 0 0 0;white-space:pre-wrap;word-break:break-word;'>{escape(text)}</pre>"

        "</div>"

    )



def refresh_history_widget():

    history_html.value = load_history_html()



def load_state():

    if PROCESS_FILE.exists():

        try:

            return json.loads(PROCESS_FILE.read_text())

        except json.JSONDecodeError:

            append_history('Process metadata unreadable; defaulting to empty state.')

    return {}



def save_state(state):

    PROCESS_FILE.write_text(json.dumps(state, indent=2))



def is_running(pid):

    if not pid:

        return False

    try:

        os.kill(pid, 0)

        return True

    except OSError:

        return False



def format_duration(seconds):

    if seconds is None:

        return '—'

    seconds = int(max(0, seconds))

    minutes, secs = divmod(seconds, 60)

    hours, minutes = divmod(minutes, 60)

    days, hours = divmod(hours, 24)

    if days:

        return f'{days}d {hours:02}:{minutes:02}:{secs:02}'

    return f'{hours:02}:{minutes:02}:{secs:02}'



def load_runtime_state():

    state = load_state()

    url = URL_FILE.read_text().strip() if URL_FILE.exists() else ''

    return state, url



def perform_health_checks(state, url, force_remote=False):

    now = time.time()

    for name in ('cloudflared', 'comfyui'):

        entry = state.get(name, {})

        pid = entry.get('pid')

        running = is_running(pid)

        started_at = entry.get('started_at')

        uptime = format_duration(now - started_at) if running and started_at else '—'

        snapshot = health_snapshot[name]

        snapshot.update({

            'running': running,

            'pid': pid,

            'uptime': uptime,

            'last_checked': now,

        })

    if state.get('comfyui'):

        if requests is None:

            health_snapshot['comfyui'].update({

                'local_ok': None,

                'local_latency': None,

                'local_status_code': None,

                'local_error': 'requests package unavailable',

            })

        else:

            try:

                start = time.perf_counter()

                resp = requests.get('http://127.0.0.1:8188/', timeout=2)

                latency = round((time.perf_counter() - start) * 1000, 1)

                health_snapshot['comfyui'].update({

                    'local_ok': resp.status_code < 500,

                    'local_latency': latency,

                    'local_status_code': resp.status_code,

                    'local_error': None,

                })

            except requests.RequestException as exc:

                health_snapshot['comfyui'].update({

                    'local_ok': False,

                    'local_latency': None,

                    'local_status_code': None,

                    'local_error': abbreviate(exc),

                })

    else:

        health_snapshot['comfyui'].update({

            'local_ok': None,

            'local_latency': None,

            'local_status_code': None,

            'local_error': None,

        })

    should_probe_remote = False

    if url and requests is not None:

        last_probe = health_snapshot['cloudflared'].get('remote_checked_at') or 0

        if force_remote or (now - last_probe) >= REMOTE_CHECK_INTERVAL:

            should_probe_remote = True

    if url and requests is None:

        for name in ('cloudflared', 'comfyui'):

            health_snapshot[name].update({

                'remote_ok': None,

                'remote_latency': None,

                'remote_status_code': None,

                'remote_error': 'requests package unavailable',

            })

    elif should_probe_remote:

        try:

            start = time.perf_counter()

            resp = requests.get(

                url,

                timeout=4,

                headers={'User-Agent': 'ComfyUI-Status-Probe'}

            )

            latency = round((time.perf_counter() - start) * 1000, 1)

            ok = resp.status_code < 500

            for name in ('cloudflared', 'comfyui'):

                health_snapshot[name].update({

                    'remote_ok': ok,

                    'remote_latency': latency,

                    'remote_status_code': resp.status_code,

                    'remote_error': None,

                    'remote_checked_at': now,

                })

        except requests.RequestException as exc:

            for name in ('cloudflared', 'comfyui'):

                health_snapshot[name].update({

                    'remote_ok': False,

                    'remote_latency': None,

                    'remote_status_code': None,

                    'remote_error': abbreviate(exc),

                    'remote_checked_at': now,

                })

    elif not url:

        for name in ('cloudflared', 'comfyui'):

            health_snapshot[name].update({

                'remote_ok': None,

                'remote_latency': None,

                'remote_status_code': None,

                'remote_error': None,

            })



def compose_status(snapshot, service_name):

    running = snapshot.get('running')

    remote_ok = snapshot.get('remote_ok')

    local_ok = snapshot.get('local_ok')

    if not running:

        return ('Offline', '#f87171')

    if service_name == 'comfyui' and local_ok is False:

        return ('Degraded', '#f97316')

    if remote_ok is False:

        return ('Degraded', '#f97316')

    if remote_ok is True:

        return ('Serving', '#34d399') if service_name == 'comfyui' else ('Active', '#34d399')

    if service_name == 'comfyui' and local_ok is True:

        return ('Running', '#60a5fa')

    return ('Checking', '#60a5fa')



def build_service_card(title, snapshot, service_name):

    status_text, status_color = compose_status(snapshot, service_name)

    pid_text = snapshot.get('pid') or '—'

    uptime_text = snapshot.get('uptime', '—')

    remote_ok = snapshot.get('remote_ok')

    remote_latency = snapshot.get('remote_latency')

    remote_status = snapshot.get('remote_status_code')

    remote_error = snapshot.get('remote_error')

    remote_checked_at = snapshot.get('remote_checked_at')

    if remote_ok is True:

        latency_text = f'{remote_latency} ms' if remote_latency is not None else '—'

        remote_line = f'Tunnel check: OK ({remote_status}) - {latency_text}'

    elif remote_ok is False:

        remote_line = f"Tunnel check: error - {escape(remote_error or 'unreachable')}"

    elif remote_ok is None:

        remote_line = 'Tunnel check: pending...'

    else:

        remote_line = 'Tunnel check: pending...'

    if remote_checked_at:

        remote_line += f" (at {time.strftime('%H:%M:%S', time.localtime(remote_checked_at))})"

    details = [

        f'PID: {pid_text}',

        f'Uptime: {uptime_text}',

        remote_line,

    ]

    if service_name == 'comfyui':

        local_ok = snapshot.get('local_ok')

        if local_ok is True:

            latency = snapshot.get('local_latency')

            latency_text = f'{latency} ms' if latency is not None else '—'

            details.append(f"Local API: OK ({snapshot.get('local_status_code')}) - {latency_text}")

        elif local_ok is False:

            details.append(f"Local API: error - {escape(snapshot.get('local_error') or 'unreachable')}")

        else:

            details.append('Local API: probing...')

    last_sample = snapshot.get('last_checked')

    if last_sample:

        details.append(f"Last sample: {time.strftime('%H:%M:%S', time.localtime(last_sample))}")

    detail_html = ''.join(f"<div>{escape(item)}</div>" for item in details)

    return (

        "<div style='background:#0b1120;border:1px solid #1f2937;border-radius:14px;padding:16px;color:#e2e8f0;font-family:\"Segoe UI\",Arial;'>"

        "<div style='display:flex;justify-content:space-between;align-items:center;'>"

        f"<span style='font-size:12px;letter-spacing:0.18em;color:#60a5fa;text-transform:uppercase;'>{escape(title)}</span>"

        f"<span style='background:{status_color};color:#0f172a;padding:4px 12px;border-radius:999px;font-weight:600;font-size:12px;'>{status_text}</span>"

        "</div>"

        f"<div style='margin-top:14px;font-family:Menlo,monospace;font-size:12px;line-height:1.6;'>{detail_html}</div>"

        "</div>"

    )



def update_status_view(state, url):

    cloud_card = build_service_card('Cloudflared Tunnel', health_snapshot['cloudflared'], 'cloudflared')

    comfy_card = build_service_card('ComfyUI Server', health_snapshot['comfyui'], 'comfyui')

    status_html.value = (

        "<div style='display:grid;grid-template-columns:repeat(auto-fit,minmax(260px,1fr));gap:16px;'>"

        f"{cloud_card}{comfy_card}"

        "</div>"

    )

    if url:

        safe_url = escape(url, quote=True)

        remote_snapshot = health_snapshot['cloudflared']

        last_probe = remote_snapshot.get('remote_checked_at')

        latency = remote_snapshot.get('remote_latency')

        latency_text = f'{latency} ms' if latency is not None else '—'

        if remote_snapshot.get('remote_ok') is True:

            remote_caption = f"Tunnel healthy - {latency_text}"

        elif remote_snapshot.get('remote_ok') is False:

            remote_caption = f"Tunnel check failed - {escape(remote_snapshot.get('remote_error') or 'error')}"

        else:

            remote_caption = 'Remote check pending...'

        probe_info = (

            f"Last remote check at {time.strftime('%H:%M:%S', time.localtime(last_probe))}" if last_probe else

            'Remote check not yet completed.'

        )

        url_html.value = (

            "<div style='background:#0b1120;border:1px solid #1f2937;border-radius:14px;padding:16px;color:#e2e8f0;font-family:\"Segoe UI\",Arial;'>"

            "<div style='font-size:12px;letter-spacing:0.14em;color:#facc15;'>PUBLIC ENDPOINT</div>"

            f"<div style='margin-top:10px;font-family:Menlo,monospace;font-size:13px;'>{escape(url)}</div>"

            f"<div style='margin-top:8px;font-size:12px;color:#94a3b8;'>{escape(probe_info)}</div>"

            "</div>"

        )

        public_link_html.value = (

            "<div style='margin-top:12px;display:flex;flex-wrap:wrap;gap:12px;align-items:center;'>"

            f"<a href='{safe_url}' target='_blank' style='background:#2563eb;color:#ffffff;padding:10px 18px;border-radius:10px;font-weight:600;text-decoration:none;'>Open ComfyUI</a>"

            f"<span style='font-size:12px;color:#94a3b8;'>{escape(remote_caption)}</span>"

            "</div>"

        )

    else:

        url_html.value = (

            "<div style='background:#0b1120;border:1px solid #1f2937;border-radius:14px;padding:16px;color:#94a3b8;font-family:\"Segoe UI\",Arial;'>"

            "<div style='font-size:12px;letter-spacing:0.14em;color:#facc15;'>PUBLIC ENDPOINT</div>"

            "<div style='margin-top:10px;font-size:13px;'>Tunnel URL not yet available.</div>"

            "</div>"

        )

        public_link_html.value = (

            "<div style='margin-top:12px;font-size:12px;color:#6b7280;'>Waiting for Cloudflared to publish a tunnel URL...</div>"

        )



def update_heartbeat():

    timestamp = time.strftime('%H:%M:%S')

    mode = 'Auto-refresh active' if auto_refresh_toggle.value else 'Auto-refresh paused'

    interval = f" - interval {AUTO_REFRESH_INTERVAL}s" if auto_refresh_toggle.value else ''

    heartbeat_html.value = (

        "<div style='background:#08111f;border:1px solid #1f2937;border-radius:12px;padding:10px 14px;color:#cbd5f5;font-family:\"Segoe UI\",Arial;font-size:12px;'>"

        f"{mode}{interval} - last update {timestamp}"

        "</div>"

    )

    dashboard_last_update['timestamp'] = time.time()



def update_dashboard(force_remote=False):

    state, url = load_runtime_state()

    perform_health_checks(state, url, force_remote=force_remote)

    update_status_view(state, url)

    update_heartbeat()



def log_action(message):

    line = append_history(message)

    with console:

        print(line)

    refresh_history_widget()



def stop_process(name, verbose=True):

    state = load_state()

    entry = state.get(name)

    if not entry:

        if verbose:

            log_action(f'{name} is not tracked.')

        return False

    pid = entry.get('pid')

    if pid and is_running(pid):

        try:

            os.killpg(pid, signal.SIGTERM)

        except ProcessLookupError:

            pass

        except OSError:

            try:

                os.kill(pid, signal.SIGTERM)

            except OSError:

                pass

        time.sleep(1)

    state.pop(name, None)

    save_state(state)

    snapshot = health_snapshot.get(name)

    if snapshot:

        snapshot.update({'running': False, 'pid': None, 'uptime': '—'})

    if verbose:

        log_action(f'{name} stopped.')

    update_dashboard(force_remote=True)

    return True



def start_process(name):

    state = load_state()

    entry = state.get(name)

    if not entry:

        raise RuntimeError(f'{name} has no stored metadata. Rerun Cell 2 to launch it.')

    cmd = entry.get('cmd')

    cwd = entry.get('cwd')

    log_path = Path(entry.get('log', ''))

    env_overrides = entry.get('env', {})

    if not cmd or not log_path:

        raise RuntimeError(f'Missing metadata for {name}; rerun Cell 2.')

    log_path.parent.mkdir(parents=True, exist_ok=True)

    if name == 'cloudflared':

        log_path.unlink(missing_ok=True)

    log_file = open(log_path, 'ab')

    env = os.environ.copy()

    env.update(env_overrides)

    proc = subprocess.Popen(

        cmd,

        stdout=log_file,

        stderr=subprocess.STDOUT,

        cwd=cwd,

        env=env,

        preexec_fn=os.setsid,

    )

    entry['pid'] = proc.pid

    entry['started_at'] = time.time()

    state[name] = entry

    save_state(state)

    snapshot = health_snapshot.get(name)

    if snapshot:

        snapshot.update({'running': True, 'pid': proc.pid, 'uptime': '00:00:00'})

    log_action(f'{name} restarted (pid {proc.pid}).')

    update_dashboard(force_remote=True)

    return proc.pid



def fetch_tunnel_url(timeout=120):

    if not CLOUDFLARED_LOG.exists():

        return None

    pattern = re.compile(r'https://[-\w.]+\.trycloudflare\.com')

    deadline = time.time() + timeout

    while time.time() < deadline:

        log_text = CLOUDFLARED_LOG.read_text(errors='ignore')

        match = pattern.search(log_text)

        if match:

            url = match.group(0)

            URL_FILE.write_text(url)

            os.environ['COMFY_URL'] = url

            append_history(f'Tunnel URL updated to {url}')

            refresh_history_widget()

            update_dashboard(force_remote=True)

            return url

        time.sleep(1.5)

    return None



def start_tail():

    tail_stop_event.clear()

    def _tail():

        position = 0

        while not tail_stop_event.is_set():

            if COMFY_LOG.exists():

                with open(COMFY_LOG, 'r', errors='ignore') as fh:

                    fh.seek(position)

                    chunk = fh.read()

                    if chunk:

                        position += len(chunk)

                        with log_stream:

                            print(chunk, end='')

            time.sleep(1.5)

    thread = threading.Thread(target=_tail, daemon=True)

    thread.start()

    return thread



def toggle_tail(change):

    global tail_thread

    if change['new']:

        log_stream.clear_output()

        tail_thread = start_tail()

        toggle_button.style.button_color = '#0ea5e9'

        log_action('Log streaming enabled.')

    else:

        tail_stop_event.set()

        if tail_thread and tail_thread.is_alive():

            tail_thread.join(timeout=0.2)

        tail_thread = None

        toggle_button.style.button_color = '#1f2937'

        log_action('Log streaming paused.')



def start_auto_refresh():

    global auto_refresh_thread

    if auto_refresh_thread and auto_refresh_thread.is_alive():

        return

    auto_refresh_stop_event.clear()

    def _loop():

        while not auto_refresh_stop_event.is_set():

            try:

                update_dashboard()

            except Exception as exc:

                with console:

                    print(f'[auto-refresh] {exc}')

            time.sleep(AUTO_REFRESH_INTERVAL)

    auto_refresh_thread = threading.Thread(target=_loop, daemon=True)

    auto_refresh_thread.start()



def stop_auto_refresh():

    global auto_refresh_thread

    auto_refresh_stop_event.set()

    if auto_refresh_thread and auto_refresh_thread.is_alive():

        auto_refresh_thread.join(timeout=0.3)

    auto_refresh_thread = None



def refresh_auto_toggle_style(active):

    if active:

        auto_refresh_toggle.style.button_color = '#2563eb'

        auto_refresh_toggle.icon = 'sync'

        auto_refresh_toggle.description = 'Auto-refresh status'

    else:

        auto_refresh_toggle.style.button_color = '#1f2937'

        auto_refresh_toggle.icon = 'play'

        auto_refresh_toggle.description = 'Enable auto-refresh'



def on_auto_refresh_change(change):

    active = change['new']

    refresh_auto_toggle_style(active)

    if active:

        start_auto_refresh()

        update_dashboard(force_remote=True)

    else:

        stop_auto_refresh()

        update_heartbeat()



try:

    enable_widget_autoscroll

except NameError:

    from IPython.display import Javascript
'''
    AUTO_SCROLL_JS = "

(function() {{

    const className = {class_name_json};

    const attach = (root) => {{

        const target = root.querySelector('.output_scroll') || root;

        if (!target) {{

            return;

        }}

        if (target.dataset.autoScrollBound === 'true') {{

            target.scrollTop = target.scrollHeight;

            return;

        }}

        target.dataset.autoScrollBound = 'true';

        const scroll = () => {{

            target.scrollTop = target.scrollHeight;

        }};

        const observer = new MutationObserver(() => requestAnimationFrame(scroll));

        observer.observe(target, {{ childList: true, subtree: true }});

        scroll();

    }};

    const ensure = () => {{

        const nodes = document.querySelectorAll('.' + className);

        if (!nodes.length) {{

            requestAnimationFrame(ensure);

            return;

        }}

        nodes.forEach(attach);

    }};

    ensure();

}})();

"



    def enable_widget_autoscroll(widget):

        class_name = f'auto-scroll-{widget.model_id}'

        widget.add_class(class_name)

        snippet = AUTO_SCROLL_JS.replace('{class_name_json}', json.dumps(class_name))

        display(Javascript(snippet))

'''

history_html = widgets.HTML()

heartbeat_html = widgets.HTML()

status_html = widgets.HTML()

url_html = widgets.HTML()

public_link_html = widgets.HTML()

console = widgets.Output(layout={'border': '1px solid #1f1f1f', 'padding': '8px', 'max_height': '220px', 'overflow_y': 'auto', 'background': '#040404', 'color': '#d8dee9', 'font_family': 'Menlo,Arial'})

#enable_widget_autoscroll(console)

log_stream = widgets.Output(layout={'border': '1px solid #1f1f1f', 'padding': '6px', 'height': '240px', 'overflow_y': 'auto', 'background': '#050505', 'color': '#cbd5f5', 'font_family': 'Menlo,monospace'})

#enable_widget_autoscroll(log_stream)



toggle_button = widgets.ToggleButton(value=True, description='Stream comfyui.log', icon='line-chart', layout=widgets.Layout(width='220px', height='42px', margin='0 0 12px 0'))

toggle_button.style.button_color = '#0ea5e9'

toggle_button.observe(toggle_tail, names='value')



stop_cloud_btn = widgets.Button(description='Stop Cloudflared', icon='stop', layout=widgets.Layout(height='46px'), style=widgets.ButtonStyle(button_color='#dc2626'))

restart_cloud_btn = widgets.Button(description='Restart Cloudflared', icon='refresh', layout=widgets.Layout(height='46px'), style=widgets.ButtonStyle(button_color='#f97316'))

stop_comfy_btn = widgets.Button(description='Stop ComfyUI', icon='stop', layout=widgets.Layout(height='46px'), style=widgets.ButtonStyle(button_color='#dc2626'))

restart_comfy_btn = widgets.Button(description='Restart ComfyUI', icon='refresh', layout=widgets.Layout(height='46px'), style=widgets.ButtonStyle(button_color='#f97316'))

clean_shutdown_btn = widgets.Button(description='Clean Shutdown (both)', icon='power-off', layout=widgets.Layout(height='46px'), style=widgets.ButtonStyle(button_color='#1f2937'))

history_refresh_btn = widgets.Button(description='Refresh Timeline', icon='history', layout=widgets.Layout(height='46px'), style=widgets.ButtonStyle(button_color='#2563eb'))

force_health_btn = widgets.Button(description='Force Health Check', icon='heartbeat', layout=widgets.Layout(width='220px', height='42px'), style=widgets.ButtonStyle(button_color='#6366f1'))



auto_refresh_toggle = widgets.ToggleButton(value=True, description='Auto-refresh status', icon='sync', layout=widgets.Layout(width='220px', height='42px'))

refresh_auto_toggle_style(True)

auto_refresh_toggle.observe(on_auto_refresh_change, names='value')



def handle_stop_cloud(_):

    stop_process('cloudflared')



def handle_restart_cloud(_):

    try:

        stop_process('cloudflared', verbose=False)

        pid = start_process('cloudflared')

        url = fetch_tunnel_url()

        if url:

            log_action(f'Current URL: {url}')

        else:

            log_action('Waiting for tunnel URL timed out.')

    except RuntimeError as exc:

        log_action(str(exc))



def handle_stop_comfy(_):

    stop_process('comfyui')



def handle_restart_comfy(_):

    try:

        stop_process('comfyui', verbose=False)

        pid = start_process('comfyui')

        log_action(f'ComfyUI restarted with pid {pid}.')

    except RuntimeError as exc:

        log_action(str(exc))



def handle_clean_shutdown(_):

    any_stopped = False

    any_stopped |= stop_process('cloudflared', verbose=False)

    any_stopped |= stop_process('comfyui', verbose=False)

    if any_stopped:

        log_action('Both services stopped.')

    else:

        log_action('Nothing was running.')

    update_dashboard(force_remote=True)



def handle_history_refresh(_):

    refresh_history_widget()

    log_action('Timeline refreshed.')



def handle_force_health(_):

    update_dashboard(force_remote=True)

    log_action('Manual health check executed.')



stop_cloud_btn.on_click(handle_stop_cloud)

restart_cloud_btn.on_click(handle_restart_cloud)

stop_comfy_btn.on_click(handle_stop_comfy)

restart_comfy_btn.on_click(handle_restart_comfy)

clean_shutdown_btn.on_click(handle_clean_shutdown)

history_refresh_btn.on_click(handle_history_refresh)

force_health_btn.on_click(handle_force_health)



controls_grid = widgets.GridBox(

    children=[

        restart_comfy_btn,

        stop_comfy_btn,

        restart_cloud_btn,

        stop_cloud_btn,

        clean_shutdown_btn,

        history_refresh_btn,

    ],

    layout=widgets.Layout(grid_template_columns='repeat(auto-fit, minmax(210px, 1fr))', grid_gap='12px', margin='16px 0')

)



dashboard = widgets.VBox([

    history_html,

    heartbeat_html,

    status_html,

    url_html,

    public_link_html,

    widgets.HBox([auto_refresh_toggle, force_health_btn], layout=widgets.Layout(flex_flow='row wrap', gap='12px', margin='12px 0 0 0')),

    controls_grid,

    console,

    widgets.VBox([toggle_button, log_stream], layout=widgets.Layout(margin='12px 0 0 0')),

])



refresh_history_widget()

update_dashboard(force_remote=True)

display(dashboard)



if auto_refresh_toggle.value:

    start_auto_refresh()



toggle_tail({'new': toggle_button.value})

log_action('Control panel ready.')

In [20]:
tail -n 200 /content/drive/MyDrive/comfyui_env/logs/comfyui.log | sed -n '1,200p'
tail -n 400 /content/drive/MyDrive/comfyui_env/logs/comfyui.log | egrep -i 'traceback|error|exception' || true

SyntaxError: invalid syntax (ipython-input-458289241.py, line 1)

In [ ]:
# Marker: backup anchor for the dashboard (idempotent)
import ipywidgets as widgets

# Ensure a named widget exists to hold the Backup UI when present.
backup_anchor = globals().get('backup_anchor')
if backup_anchor is None:
    backup_anchor = widgets.VBox(layout=widgets.Layout(margin='12px 0 0 0'))
    globals()['backup_anchor'] = backup_anchor

# Try to insert backup_anchor into the dashboard at a stable location (after controls_grid)
try:
    if 'dashboard' in globals() and isinstance(dashboard, widgets.VBox):
        children = list(dashboard.children)
        # Prefer to place after the controls_grid if we can find it
        insert_at = None
        for i, child in enumerate(children):
            # best-effort match by object identity
            if 'controls_grid' in globals() and child is globals().get('controls_grid'):
                insert_at = i + 1
                break
        if insert_at is None:
            # fallback: try to insert before the console widget if present
            for i, child in enumerate(children):
                if 'console' in globals() and child is globals().get('console'):
                    insert_at = i
                    break
        if insert_at is None:
            insert_at = len(children)
        # idempotent insertion
        if backup_anchor not in children:
            children.insert(insert_at, backup_anchor)
            dashboard.children = tuple(children)
            try:
                log_action('Backup anchor inserted into dashboard at index ' + str(insert_at))
            except Exception:
                print('Backup anchor inserted into dashboard at index', insert_at)
except Exception as e:
    print('Could not insert backup anchor into dashboard (ok to ignore):', e)

# Expose for other cells to detect
globals()['backup_anchor'] = backup_anchor
display(widgets.HTML('<div style="font-size:12px;color:#94a3b8;">Backup anchor ready</div>'))


In [7]:
# Health-probe helpers: explicit manual probe and optional background auto-probe
import threading
import time
from IPython.display import display, HTML
import ipywidgets as widgets

_auto_health_thread = None
_auto_health_stop = threading.Event()


def manual_health_probe():
    """Run a one-off health probe (local + remote) and refresh dashboard views."""
    try:
        update_dashboard(force_remote=True)
        print('Manual health probe executed.')
    except Exception as exc:
        print('Manual health probe failed:', exc)


def _auto_probe_loop(interval=30):
    while not _auto_health_stop.is_set():
        try:
            update_dashboard(force_remote=True)
        except Exception as e:
            print('[auto-probe] error:', e)
        time.sleep(interval)


def start_auto_health(interval=30):
    global _auto_health_thread
    if _auto_health_thread and _auto_health_thread.is_alive():
        print('Auto health probe already running.')
        return
    _auto_health_stop.clear()
    _auto_health_thread = threading.Thread(target=_auto_probe_loop, args=(interval,), daemon=True)
    _auto_health_thread.start()
    print('Started auto health probe (interval', interval, 's)')


def stop_auto_health():
    global _auto_health_thread
    _auto_health_stop.set()
    if _auto_health_thread and _auto_health_thread.is_alive():
        _auto_health_thread.join(timeout=0.5)
    _auto_health_thread = None
    print('Stopped auto health probe.')

# Small UI to trigger manual probe and toggle auto probe
btn_manual = widgets.Button(description='Manual Health Probe', button_style='info')
btn_start = widgets.Button(description='Start Auto Probe (30s)', button_style='success')
btn_stop = widgets.Button(description='Stop Auto Probe', button_style='warning')

btn_manual.on_click(lambda *_: manual_health_probe())
btn_start.on_click(lambda *_: start_auto_health(30))
btn_stop.on_click(lambda *_: stop_auto_health())

display(widgets.HBox([btn_manual, btn_start, btn_stop]))


Manual health probe executed.


In [ ]:
# Git-based backup helpers (copy into local git repo and commit)
import subprocess
import shutil
import time
from pathlib import Path


def git_store_archive(archive_path, git_repo_path, commit_message=None, push=False):
    """Copy an existing backup archive into a local git repository under 'backups/', commit it, and optionally push.

    Notes:
    - If the target path does not contain a git repo, a new repo will be initialized.
    - Commits use a local git user.name/email if none are configured for that repo.
    - Pushing from Colab may require credentials (PAT or SSH). If push fails, the archive is still stored locally and can be pushed from another machine.
    """
    if not archive_path:
        raise ValueError('archive_path is required')
    archive = Path(archive_path)
    if not archive.exists():
        raise FileNotFoundError(str(archive))
    repo = Path(git_repo_path)
    repo.mkdir(parents=True, exist_ok=True)
    backups_dir = repo / 'backups'
    backups_dir.mkdir(parents=True, exist_ok=True)
    dest = backups_dir / archive.name
    shutil.copy2(str(archive), str(dest))

    try:
        # Initialize repo if needed
        if not (repo / '.git').exists():
            subprocess.run(['git', '-C', str(repo), 'init'], check=True)
        # Ensure local user config to allow commits
        name_proc = subprocess.run(['git', '-C', str(repo), 'config', '--get', 'user.name'], capture_output=True, text=True)
        if not name_proc.stdout.strip():
            subprocess.run(['git', '-C', str(repo), 'config', 'user.name', 'colab-backup'], check=True)
            subprocess.run(['git', '-C', str(repo), 'config', 'user.email', 'colab@example.com'], check=True)
        # Add and commit
        rel = str(dest.relative_to(repo))
        subprocess.run(['git', '-C', str(repo), 'add', rel], check=True)
        msg = commit_message or f'Backup {dest.name} ({time.strftime("%Y%m%d-%H%M%S")})'
        subprocess.run(['git', '-C', str(repo), 'commit', '-m', msg], check=True)
        if push:
            try:
                subprocess.run(['git', '-C', str(repo), 'push'], check=True)
            except Exception as exc:
                log(f'Push failed: {exc} (credentials likely required).')
                log('To push from Colab, configure credentials or use a Personal Access Token (PAT). Alternatively, push locally from a machine with repository access.')
        log(f'Archive copied and committed to {dest}')
        return str(dest)
    except subprocess.CalledProcessError as exc:
        raise RuntimeError(f'Git operation failed: {exc}')


In [ ]:
# Backup/Restore UI integrated with the control panel (appends to existing `dashboard`)
import time
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

# Load existing backup config if available
cfg = load_backup_config() if 'load_backup_config' in globals() else {'github_repo_path': ''}

# Widgets
backup_create_btn = widgets.Button(description='Create Backup', icon='archive', button_style='primary', layout=widgets.Layout(width='180px', height='40px'))
backup_list_btn = widgets.Button(description='List Backups', icon='list', button_style='', layout=widgets.Layout(width='180px', height='40px'))
restore_dropdown = widgets.Dropdown(options=[], description='Restore:', layout=widgets.Layout(width='420px'))
restore_btn = widgets.Button(description='Restore Selected', icon='upload', button_style='warning', layout=widgets.Layout(width='180px', height='40px'))

git_repo_text = widgets.Text(value=cfg.get('github_repo_path',''), description='Local git repo:', layout=widgets.Layout(width='520px'))
backup_git_btn = widgets.Button(description='Store in Git Repo', icon='code-fork', button_style='success', layout=widgets.Layout(width='180px', height='40px'))
save_cfg_btn = widgets.Button(description='Save Backup Config', icon='save', button_style='', layout=widgets.Layout(width='180px', height='40px'))

# Helper to refresh dropdown options
def update_backup_dropdown():
    try:
        archives = list_backups()
    except Exception:
        archives = []
    options = []
    for a in archives:
        ts = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(a['mtime']))
        name = Path(a['path']).name
        options.append((f"{ts} {name}", a['path']))
    if not options:
        restore_dropdown.options = []
        restore_dropdown.placeholder = 'No backups available'
    else:
        restore_dropdown.options = options
        restore_dropdown.value = options[0][1]

# Button handlers

def handle_create_backup(_):
    try:
        path = create_backup()
        log_action(f'Backup created: {path}')
        update_backup_dropdown()
    except Exception as exc:
        log_action(f'Create backup failed: {exc}')


def handle_list_backups(_):
    try:
        archives = list_backups()
        if not archives:
            log_action('No backups found.')
            return
        for a in archives:
            log_action(f"{time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(a['mtime']))} - {a['path']} ({a['size']} bytes)")
    except Exception as exc:
        log_action(f'List backups failed: {exc}')


def handle_restore_backup(_):
    try:
        archive = restore_dropdown.value
        if not archive:
            log_action('No archive selected for restore.')
            return
        result = restore_backup(archive, merge_missing=True)
        log_action(f"Restore complete: extracted {len(result.get('extracted', []))}, skipped {len(result.get('skipped', []))}")
        update_dashboard(force_remote=True)
    except Exception as exc:
        log_action(f'Restore failed: {exc}')


def handle_git_store(_):
    try:
        archive = restore_dropdown.value
        if not archive:
            # create a backup and use it
            archive = create_backup()
        repo = git_repo_text.value.strip()
        if not repo:
            log_action('Local git repo path not set. Enter a path in the text field.')
            return
        dest = git_store_archive(archive, repo, commit_message=f'Backup {Path(archive).name}')
        log_action(f'Archive stored in git repo: {dest}')
    except Exception as exc:
        log_action(f'Git store failed: {exc}')


def handle_save_cfg(_):
    try:
        cfg = load_backup_config()
    except Exception:
        cfg = dict(DEFAULT_BACKUP_CONFIG)
    cfg['github_repo_path'] = git_repo_text.value.strip()
    save_backup_config(cfg)
    log_action('Backup config saved.')

# Wire up events
backup_create_btn.on_click(handle_create_backup)
backup_list_btn.on_click(handle_list_backups)
restore_btn.on_click(handle_restore_backup)
backup_git_btn.on_click(handle_git_store)
save_cfg_btn.on_click(handle_save_cfg)

# Build widget and attach to existing dashboard (if present)
backup_controls = widgets.HBox([backup_create_btn, backup_list_btn, restore_btn, backup_git_btn, save_cfg_btn], layout=widgets.Layout(gap='12px', flex_flow='row wrap'))
backup_row = widgets.VBox([widgets.HBox([restore_dropdown, git_repo_text], layout=widgets.Layout(gap='12px')), backup_controls], layout=widgets.Layout(margin='12px 0 0 0'))
backup_box = widgets.VBox([widgets.HTML("<div style='font-weight:600;padding-bottom:6px;'>Backups & Restore</div>"), backup_row], layout=widgets.Layout(border='1px solid #123', padding='8px', border_radius='8px'))

# Prefer to insert backup_box into a dedicated backup_anchor if present; otherwise append to dashboard
try:
    update_backup_dropdown()
    if 'backup_anchor' in globals() and isinstance(backup_anchor, widgets.VBox):
        # Place the backup UI explicitly inside the anchor (safer and explicit)
        backup_anchor.children = (backup_box,)
        log_action('Backup UI placed into backup_anchor.')
    elif 'dashboard' in globals() and isinstance(dashboard, widgets.VBox):
        # Fallback: insert as the last child of the existing dashboard UI
        dashboard.children = tuple(list(dashboard.children) + [backup_box])
        log_action('Backup UI appended to dashboard.')
    else:
        display(backup_box)
        log_action('Backup UI displayed inline (no dashboard present).')
except Exception:
    try:
        display(backup_box)
        update_backup_dropdown()
        log_action('Backup UI displayed (dashboard integration skipped).')
    except Exception:
        pass


In [ ]:
# Dry-run: execute a read-only integrity snapshot and print a brief report
result = check_files_readonly()
print('Dry-run check_files_readonly() summary:')
print('- Missing targets:', result['report'].get('missing'))
print('- Snapshot captured_at:', result['snapshot'].get('captured_at'))
print('\nTip: Run this cell in Colab after mounting Drive and ensuring the notebook has run its setup cells.')


In [36]:
# PAT-based push flow: ephemeral remote push using a Personal Access Token (PAT)
import subprocess
from pathlib import Path
import threading
import ipywidgets as widgets
from IPython.display import display


def git_push_with_pat(repo_path, pat, remote_url=None, branch=None):
    """Push current HEAD to remote using a PAT without storing credentials.

    - repo_path: local repo path (Path or str)
    - pat: Personal Access Token (string). This function will not store the token persistently but it will appear in subprocess args while running.
    - remote_url: https remote URL for the target repo (e.g. https://github.com/owner/repo.git). If omitted, 'origin' remote is used.
    - branch: branch name to push to (defaults to current branch or 'main').

    SECURITY: The PAT will briefly appear in process arguments and in git's remote config while the temporary remote exists.
    Use ephemeral tokens and rotate them. Do not print or save the PAT. This helper removes the temporary remote after the push.
    """
    repo = Path(repo_path)
    if not repo.exists():
        raise FileNotFoundError(f'Repo path does not exist: {repo}')

    # Determine remote name and URL
    remote_name = 'temp-pat-remote'
    if remote_url is None:
        # Use origin if available
        try:
            out = subprocess.run(['git', '-C', str(repo), 'remote', 'get-url', 'origin'], capture_output=True, text=True, check=True)
            remote_url_actual = out.stdout.strip()
        except subprocess.CalledProcessError:
            raise RuntimeError('No remote_url provided and origin not configured for repo.')
    else:
        remote_url_actual = remote_url

    if branch is None:
        try:
            out = subprocess.run(['git', '-C', str(repo), 'rev-parse', '--abbrev-ref', 'HEAD'], capture_output=True, text=True, check=True)
            branch = out.stdout.strip()
        except subprocess.CalledProcessError:
            branch = 'main'

    # Build authenticated remote URL using PAT (HTTPS)
    if remote_url_actual.startswith('https://'):
        auth_url = remote_url_actual.replace('https://', f'https://{pat}@')
    else:
        raise RuntimeError('Remote URL must be HTTPS to use PAT authentication.')

    try:
        # Add temporary remote
        subprocess.run(['git', '-C', str(repo), 'remote', 'add', remote_name, auth_url], check=True)
        # Push
        subprocess.run(['git', '-C', str(repo), 'push', remote_name, f'HEAD:{branch}'], check=True)
    finally:
        # Remove temporary remote (best-effort)
        try:
            subprocess.run(['git', '-C', str(repo), 'remote', 'remove', remote_name], check=True)
        except Exception:
            pass


def _push_with_pat_background(repo_path, pat, remote_url=None, branch=None, output_widget=None):
    try:
        if output_widget:
            with output_widget:
                print('Starting PAT push...')
        git_push_with_pat(repo_path, pat, remote_url=remote_url, branch=branch)
        if output_widget:
            with output_widget:
                print('PAT push completed successfully.')
    except Exception as exc:
        if output_widget:
            with output_widget:
                print('PAT push failed:', exc)


# Small secure UI to accept a PAT temporarily and perform a push without storing token
_pat_info = widgets.HTML('<div style="font-size:12px;color:#f59e0b;">Enter a Personal Access Token (PAT) for a one-time push. The token is not stored by this notebook. Use ephemeral tokens and rotate them. Avoid running this in a shared environment.</div>')
_repo_path = widgets.Text(value='', description='Repo path:', layout=widgets.Layout(width='540px'))
_remote_url = widgets.Text(value='', description='Remote URL (optional):', layout=widgets.Layout(width='540px'))
_branch = widgets.Text(value='', description='Branch (optional):', layout=widgets.Layout(width='260px'))
_pat = widgets.Password(placeholder='Enter PAT here (will not be displayed)', description='PAT:')
_pat_push_btn = widgets.Button(description='Push with PAT (one-time)', button_style='danger')
_pat_output = widgets.Output(layout={'border': '1px solid #1f1f1f', 'padding': '6px', 'height': '120px', 'overflow_y': 'auto'})


def _on_pat_push(_):
    repo = _repo_path.value.strip()
    if not repo:
        with _pat_output:
            print('Repo path is required.')
        return
    pat = _pat.value
    if not pat:
        with _pat_output:
            print('PAT required.')
        return
    remote = _remote_url.value.strip() or None
    branch = _branch.value.strip() or None
    # Clear the password field immediately after read
    _pat.value = ''
    # Run push in background
    th = threading.Thread(target=_push_with_pat_background, args=(repo, pat, remote, branch, _pat_output), daemon=True)
    th.start()

_pat_push_btn.on_click(_on_pat_push)

# Display the UI
display(widgets.VBox([_pat_info, _repo_path, _remote_url, widgets.HBox([_branch, _pat]), _pat_push_btn, _pat_output]))

# Security note
print('PAT push UI ready. The PAT is not stored persistently; it is used only for the immediate push.')

# Expose a helper function programmatically if needed
globals()['git_push_with_pat'] = git_push_with_pat


PAT push UI ready. The PAT is not stored persistently; it is used only for the immediate push.


In [37]:
# Example: copy a created archive into a local git repo and push from your local machine
from pathlib import Path

# Example usage in Colab (ephemeral, will only commit locally inside the provided repo path):
example_archive = None
try:
    # if a backup exists in the dropdown, use it; otherwise create one
    archives = list_backups()
    if archives:
        example_archive = archives[0]['path']
    else:
        example_archive = create_backup()
    # Example local repo path on Drive (user-provided). This is a local git repo path on Drive.
    example_repo = str(Path.home() / 'comfyui_backup_repo') if 'Path' in globals() else ''
    print('Example archive chosen:', example_archive)
    print('\nTo copy and commit this archive into a local repo from this notebook, call:')
    print("    git_store_archive(example_archive, '<path-to-local-git-repo>')")
    print('\nTo push the committed archive from your own machine (recommended), run these commands on your local machine where you have credentials:')
    print('  cd /path/to/local/git/repo')
    print('  git pull origin main  # ensure your local copy is up-to-date')
    print('  git push origin main  # push the committed backup to the remote GitHub repository')
    print('\nIf you prefer to push directly from Colab using a PAT (ephemeral), you can run:')
    print("    git_push_with_pat('<local-repo-path>', '<YOUR_PAT>', remote_url='https://github.com/owner/repo.git')")
    print('\nNote: Using PAT from Colab is less secure; prefer pushing from a machine you control.')
except Exception as e:
    print('Example setup skipped (could not locate or create an archive):', e)


KeyboardInterrupt: 

In [21]:
# Move existing Backup UI into a collapsible Accordion within the dashboard (idempotent)
import ipywidgets as widgets
from IPython.display import display

try:
    if 'backup_box' not in globals():
        log_action('backup_box not found in globals; run the Backup UI creation cell first.')
    else:
        if 'backup_accordion' in globals():
            log_action('Backup accordion already present; skipping creation.')
        else:
            backup_accordion = widgets.Accordion(children=[backup_box])
            backup_accordion.set_title(0, 'Backups & Restore (click to expand)')
            backup_accordion.layout = widgets.Layout(width='100%')
            # Try to replace any standalone backup_box in dashboard
            try:
                if 'dashboard' in globals():
                    children = list(dashboard.children)
                    # Remove any exact references to backup_box
                    children = [c for c in children if c is not backup_box]
                    # Append accordion at the end
                    children.append(backup_accordion)
                    dashboard.children = tuple(children)
                    log_action('Backup UI moved into collapsible accordion in control panel.')
                else:
                    display(backup_accordion)
                    log_action('Dashboard not found; displayed backup accordion standalone.')
            except Exception as exc:
                display(backup_accordion)
                log_action(f'Failed to integrate accordion into dashboard: {exc}')
except Exception as e:
    try:
        log_action(f'Error while creating backup accordion: {e}')
    except Exception:
        print('Error while creating backup accordion:', e)


In [ ]:
# Safe, robust enable_widget_autoscroll implementation
# This replaces the fragile version that used string-format and caused the cell to fail.
from IPython.display import Javascript, display
import json


def enable_widget_autoscroll(widget, max_tries=5):
    """Attach an auto-scroll behavior to a widget's output area in a safe way.

    - Adds a unique CSS class derived from widget.model_id.
    - Injects a small JS snippet that waits for the element to exist, then attaches a
      MutationObserver that keeps the scroll at the bottom when new content arrives.
    - This implementation avoids using Python string templating inside large JS strings
      and uses json.dumps to safely embed the class name.
    - It tolerates environments where the frontend JS API is not available (e.g., headless tests).
    """
    try:
        class_name = f'auto-scroll-{widget.model_id}'
    except Exception:
        # If widget has no model_id (rare), fall back to a randomized name
        import uuid
        class_name = f'auto-scroll-{uuid.uuid4().hex[:8]}'

    try:
        widget.add_class(class_name)
    except Exception:
        # Some widgets may not support add_class in this environment; ignore
        pass

    # Small JS that will attach a MutationObserver to the matching node(s)
    js = f"""
(function() {{
  const className = {json.dumps(class_name)};
  function attachToElement(el) {{
    try {{
      if (el.dataset && el.dataset._autoScrollBound === 'true') {{
        el.scrollTop = el.scrollHeight;
        return;
      }}
      el.dataset._autoScrollBound = 'true';
      const scroll = () => {{ el.scrollTop = el.scrollHeight; }};
      const observer = new MutationObserver(scroll);
      observer.observe(el, {{ childList: true, subtree: true }});
      scroll();
    }} catch (e) {{ console.warn('attachToElement error', e); }}
  }}
  function findAndAttach() {{
    const nodes = document.querySelectorAll('.' + className);
    if (!nodes || nodes.length === 0) {{
      // Not present yet; try again shortly
      setTimeout(findAndAttach, 250);
      return;
    }}
    nodes.forEach((n) => {{
      // Prefer .output_scroll descendant if present
      const outputScroll = n.querySelector('.output_scroll') || n;
      attachToElement(outputScroll);
    }});
  }}
  // Kick off
  findAndAttach();
}})();
"""

    try:
        display(Javascript(js))
    except Exception as exc:
        # If display or JS injection isn't available (non-notebook env), ignore silently
        try:
            print('Autoscroll JS injection skipped (no frontend):', exc)
        except Exception:
            pass

    return class_name

# Informational message (helpful during development)
try:
    log_action('Patched enable_widget_autoscroll with safer implementation.')
except Exception:
    print('Patched enable_widget_autoscroll with safer implementation.')


In [ ]:
# Auto-enable autoscroll for control-panel widgets and provide a cleanup helper
# This cell is idempotent and safe to re-run.

_autoscroll_classes = getattr(globals(), '_autoscroll_classes', set())


def _attach_autoscroll_if_available():
    """Attach autoscroll to common control-panel widgets if they exist in globals()."""
    attached = []
    for name in ('console', 'log_stream', 'history_html'):
        widget = globals().get(name)
        if widget is None:
            continue
        try:
            cls = enable_widget_autoscroll(widget)
            _autoscroll_classes.add(cls)
            attached.append((name, cls))
        except Exception as exc:
            try:
                log_action(f'Autoscroll attach failed for {name}: {exc}')
            except Exception:
                print(f'Autoscroll attach failed for {name}: {exc}')
    return attached


def cleanup_autoscroll():
    """Remove autoscroll classes from widgets and attempt to signal the frontend to stop observers.

    Notes:
    - We cannot directly kill MutationObservers created in JS from Python, but we can remove the CSS class
      and request the frontend to clear the data attribute used to mark bound nodes. Re-running enable will
      reattach correctly.
    """
    removed = []
    for name in ('console', 'log_stream', 'history_html'):
        widget = globals().get(name)
        if widget is None:
            continue
        try:
            # Remove class if supported
            for cls in list(_autoscroll_classes):
                try:
                    widget.remove_class(cls)
                    removed.append((name, cls))
                    _autoscroll_classes.discard(cls)
                except Exception:
                    pass
        except Exception as exc:
            print('Cleanup issue for', name, exc)
    # Ask frontend to clear data attributes from any nodes matching remaining classes
    if _autoscroll_classes:
        js = f"""
(function() {{
  const classes = {json.dumps(list(_autoscroll_classes))};
  classes.forEach((cls) => {{
    const nodes = document.querySelectorAll('.' + cls);
    nodes.forEach((n) => {{
      try {{
        if (n.dataset) {{ delete n.dataset._autoScrollBound; }}
      }} catch (e) {{ console.warn('cleanup_autoscroll error', e); }}
    }});
  }});
}})();
"""
        try:
            display(Javascript(js))
        except Exception:
            pass
    return removed


# Attach automatically now (idempotent)
attached = _attach_autoscroll_if_available()
if attached:
    try:
        log_action(f'Autoscroll attached to: {attached}')
    except Exception:
        print('Autoscroll attached to:', attached)
else:
    try:
        log_action('No matching control-panel widgets were present for autoscroll attach.')
    except Exception:
        print('No matching control-panel widgets were present for autoscroll attach.')

# Expose cleanup helper at top-level for convenience
globals()['cleanup_autoscroll'] = cleanup_autoscroll


In [ ]:
# Interactive terminal + quick action widgets (idempotent)
# Provides a small CLI input, verbose output area, and top widgets for common ComfyUI/Manager actions.

import shlex
import subprocess
import threading
import queue
import time
import ipywidgets as widgets
from IPython.display import display, HTML

# Reuse console output widget if present, otherwise create a local verbose output
_verbose_out = globals().get('interactive_terminal_output')
if _verbose_out is None:
    _verbose_out = widgets.Output(layout={'border': '1px solid #1f1f1f', 'padding': '6px', 'height': '320px', 'overflow_y': 'auto', 'background': '#000', 'color': '#e6eef8', 'font_family': 'Menlo,monospace'})
    globals()['interactive_terminal_output'] = _verbose_out

# Command input and run button
_cmd_input = widgets.Text(placeholder='Enter shell command (e.g. ls -la /content) or use shortcuts below', layout=widgets.Layout(width='72%'))
_run_btn = widgets.Button(description='Run', button_style='primary', icon='play', layout=widgets.Layout(width='84px'))

# Quick action buttons for ComfyUI/Manager API and CLI
_status_btn = widgets.Button(description='Check Server', icon='heartbeat', button_style='info')
_restart_btn = widgets.Button(description='Restart ComfyUI', icon='refresh', button_style='warning')
_update_btn = widgets.Button(description='Update Repos', icon='download', button_style='')
_backup_btn = widgets.Button(description='Create Backup', icon='archive', button_style='success')

# Dropdown for predefined commands
_preset = widgets.Dropdown(options=[('', ''), ('Tail Comfy log', f'tail -n 200 {COMFY_LOG}'), ('Show cloudflared log', f'tail -n 200 {CLOUDFLARED_LOG}'), ('List backups', 'ls -la ' + str(BACKUP_DIR)), ('Show session log', 'tail -n 200 ' + str(SESSION_LOG))], description='Presets:', layout=widgets.Layout(width='65%'))

# Internal worker queue
_cmd_queue = globals().get('_cmd_queue')
if _cmd_queue is None:
    _cmd_queue = queue.Queue()
    globals()['_cmd_queue'] = _cmd_queue


def _worker_loop():
    while True:
        try:
            item = _cmd_queue.get()
            if item is None:
                break
            cmd, caller = item
            with _verbose_out:
                print('\n$ ' + cmd)
            try:
                # Use shell for convenience, but keep it verbose
                proc = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
                for line in proc.stdout:
                    with _verbose_out:
                        print(line, end='')
                proc.wait()
                with _verbose_out:
                    print(f'-- exit {proc.returncode}\n')
            except Exception as exc:
                with _verbose_out:
                    print('Command failed:', exc)
        except Exception:
            time.sleep(0.1)


_worker = globals().get('_terminal_worker')
if _worker is None:
    _worker = threading.Thread(target=_worker_loop, daemon=True)
    _worker.start()
    globals()['_terminal_worker'] = _worker


def run_command_from_input(_cmd):
    cmd = _cmd.strip()
    if not cmd:
        return
    _cmd_queue.put((cmd, 'manual'))


def on_run_clicked(_):
    cmd = _cmd_input.value or _preset.value
    if not cmd:
        with _verbose_out:
            print('No command provided (use input or select a preset).')
        return
    run_command_from_input(cmd)


def on_preset_change(change):
    if change['new']:
        run_command_from_input(change['new'])


# Quick actions implementation

def check_server(_):
    # Try local API first, fallback to checking process
    try:
        import requests
        with _verbose_out:
            print('Checking local ComfyUI HTTP endpoint...')
        try:
            r = requests.get('http://127.0.0.1:8188/', timeout=2)
            with _verbose_out:
                print(f'Local HTTP: status {r.status_code}, len={len(r.text) if r.text else 0}')
        except Exception as exc:
            with _verbose_out:
                print('Local HTTP probe failed:', exc)
            # Fallback: check pid state
            state = load_state() if 'load_state' in globals() else {}
            comfy = state.get('comfyui')
            if comfy:
                pid = comfy.get('pid')
                running = is_running(pid) if 'is_running' in globals() else False
                with _verbose_out:
                    print('ComfyUI process pid:', pid, 'running:', running)
            else:
                with _verbose_out:
                    print('ComfyUI metadata not present in runtime state.')
    except Exception as exc:
        with _verbose_out:
            print('Status check error:', exc)


def restart_comfy(_):
    with _verbose_out:
        print('Restarting ComfyUI...')
    try:
        stop_process('comfyui')
        pid = start_process('comfyui')
        with _verbose_out:
            print('ComfyUI restarted, pid', pid)
    except Exception as exc:
        with _verbose_out:
            print('Restart failed:', exc)


def update_repos(_):
    with _verbose_out:
        print('Updating ComfyUI and manager repositories...')
    try:
        update_comfyui()
        update_manager()
        with _verbose_out:
            print('Update finished.')
    except Exception as exc:
        with _verbose_out:
            print('Update failed:', exc)


def quick_backup(_):
    with _verbose_out:
        print('Creating backup...')
    try:
        path = create_backup()
        with _verbose_out:
            print('Backup created at', path)
    except Exception as exc:
        with _verbose_out:
            print('Backup failed:', exc)


# Wire buttons
_run_btn.on_click(on_run_clicked)
_preset.observe(on_preset_change, names='value')
_status_btn.on_click(check_server)
_restart_btn.on_click(restart_comfy)
_update_btn.on_click(update_repos)
_backup_btn.on_click(quick_backup)

# Layout
_top_row = widgets.HBox([_status_btn, _restart_btn, _update_btn, _backup_btn, _preset], layout=widgets.Layout(gap='8px'))
_cmd_row = widgets.HBox([_cmd_input, _run_btn], layout=widgets.Layout(margin='8px 0 0 0', align_items='center'))
_terminal_box = widgets.VBox([_top_row, _cmd_row, _verbose_out], layout=widgets.Layout(width='100%'))

display(HTML('<div style="font-size:12px;color:#94a3b8;margin-bottom:6px;">Interactive terminal — use presets or enter shell commands. Output is verbose.</div>'))
# Show in a dedicated area beneath the dashboard anchor if present
try:
    if 'backup_anchor' in globals() and isinstance(backup_anchor, widgets.VBox):
        # Append terminal below backup_anchor for a stable layout
        container_children = list(backup_anchor.children)
        if _terminal_box not in container_children:
            container_children.append(_terminal_box)
            backup_anchor.children = tuple(container_children)
            with _verbose_out:
                print('Interactive terminal attached below backup anchor.')
    else:
        display(_terminal_box)
except Exception as exc:
    display(_terminal_box)
    with _verbose_out:
        print('Failed to attach to backup_anchor; displayed inline. Error:', exc)

# Expose run helper
globals()['interactive_run_command'] = run_command_from_input
print('Interactive terminal ready. Use the Run button or call interactive_run_command("your command").')
